# Census Data Preparation

**Tasks:**
* Maybe delete total population from education and final dataframe

In [ ]:
import os
import zipfile

# Unzip all files
path = "drive/MyDrive/US Elections/Census Data"

for folder in os.listdir(path):
  if folder.endswith("2016"):
    zip_file = os.listdir(f"{path}/{folder}")[0]
    total_path = f"{path}/{folder}/{zip_file}"
    with zipfile.ZipFile(total_path,"r") as zip_ref:
      zip_ref.extractall(f"{path}/{folder}")

## Educational Attainment USA

The end goal is to have a dataframe which shows the perentage of people having a certain educational attainment within a given district. 

Steps:
1. Clean redundant columns✅
2. Summarize education into categories *(below highschool, highschool, college or associate and bachelor or higher)*✅
3. Include state column to better join them later with other dataframes✅
4. Clean district names so that they match the ones of my target variable dataframe✅
4. Calculate percentage values✅
5. Drop all districts which are not available in the target variable dataframe✅

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
education = pd.read_csv("drive/MyDrive/US Elections/Census Data/Educational Attainment 2016/educational_attainment_2016.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,118,119,122,123,124,125,126,127,128,129,132,133,134,135,136,137,138,139,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,162,163,164,165,168,169,170,171,174,175,176,177,180,181,182,183,186,187,188,189,192,193,194,195,228,229,230,231,234,235,236,237,238,239,240,241,242,243,244,245,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,3

In [ ]:
education.head()

,GEO_ID,NAME,S1501_C01_001E,S1501_C01_001M,S1501_C01_002E,S1501_C01_002M,S1501_C01_003E,S1501_C01_003M,S1501_C01_004E,S1501_C01_004M,S1501_C01_005E,S1501_C01_005M,S1501_C01_006E,S1501_C01_006M,S1501_C01_007E,S1501_C01_007M,S1501_C01_008E,S1501_C01_008M,S1501_C01_009E,S1501_C01_009M,S1501_C01_010E,S1501_C01_010M,S1501_C01_011E,S1501_C01_011M,S1501_C01_012E,S1501_C01_012M,S1501_C01_013E,S1501_C01_013M,S1501_C01_014E,S1501_C01_014M,S1501_C01_015E,S1501_C01_015M,S1501_C01_016E,S1501_C01_016M,S1501_C01_017E,S1501_C01_017M,S1501_C01_018E,S1501_C01_018M,S1501_C01_019E,S1501_C01_019M,S1501_C01_020E,S1501_C01_020M,S1501_C01_021E,S1501_C01_021M,S1501_C01_022E,S1501_C01_022M,S1501_C01_023E,S1501_C01_023M,S1501_C01_024E,S1501_C01_024M,S1501_C01_025E,S1501_C01_025M,S1501_C01_026E,S1501_C01_026M,S1501_C01_027E,S1501_C01_027M,S1501_C01_028E,S1501_C01_028M,S1501_C01_029E,S1501_C01_029M,S1501_C01_030E,S1501_C01_030M,S1501_C01_031E,S1501_C01_031M,S1501_C01_032E,S1501_C01_032M,S1501_C01_033E,S1501_C01_033M,S1501_C01_034E,S1501_C01_034M,S1501_C01_035E,S1501_C01_035M,S1501_C01_036E,S1501_C01_036M,S1501_C01_037E,S1501_C01_037M,S1501_C01_038E,S1501_C01_038M,S1501_C01_039E,S1501_C01_039M,S1501_C01_040E,S1501_C01_040M,S1501_C01_041E,S1501_C01_041M,S1501_C01_042E,S1501_C01_042M,S1501_C01_043E,S1501_C01_043M,S1501_C01_044E,S1501_C01_044M,S1501_C01_045E,S1501_C01_045M,S1501_C01_046E,S1501_C01_046M,S1501_C01_047E,S1501_C01_047M,S1501_C01_048E,S1501_C01_048M,S1501_C01_049E,S1501_C01_049M,S1501_C01_050E,S1501_C01_050M,S1501_C01_051E,S1501_C01_051M,S1501_C01_052E,S1501_C01_052M,S1501_C01_053E,S1501_C01_053M,S1501_C01_054E,S1501_C01_054M,S1501_C01_055E,S1501_C01_055M,S1501_C01_056E,S1501_C01_056M,S1501_C01_057E,S1501_C01_057M,S1501_C01_058E,S1501_C01_058M,S1501_C01_059E,S1501_C01_059M,S1501_C01_060E,S1501_C01_060M,S1501_C01_061E,S1501_C01_061M,S1501_C01_062E,S1501_C01_062M,S1501_C01_063E,S1501_C01_063M,S1501_C01_064E,S1501_C01_064M,S1501_C02_001E,S1501_C02_001M,S1501_C02_002E,S1501_C02_002M,S1501_C02_003E,S1501_C02_003M,S1501_C02_004E,S1501_C02_004M,S1501_C02_005E,S1501_C02_005M,S1501_C02_006E,S1501_C02_006M,S1501_C02_007E,S1501_C02_007M,S1501_C02_008E,S1501_C02_008M,S1501_C02_009E,S1501_C02_009M,S1501_C02_010E,S1501_C02_010M,S1501_C02_011E,S1501_C02_011M,S1501_C02_012E,S1501_C02_012M,S1501_C02_013E,S1501_C02_013M,S1501_C02_014E,S1501_C02_014M,S1501_C02_015E,S1501_C02_015M,S1501_C02_016E,S1501_C02_016M,S1501_C02_017E,S1501_C02_017M,S1501_C02_018E,S1501_C02_018M,S1501_C02_019E,S1501_C02_019M,S1501_C02_020E,S1501_C02_020M,S1501_C02_021E,S1501_C02_021M,S1501_C02_022E,S1501_C02_022M,S1501_C02_023E,S1501_C02_023M,S1501_C02_024E,S1501_C02_024M,S1501_C02_025E,S1501_C02_025M,S1501_C02_026E,S1501_C02_026M,S1501_C02_027E,S1501_C02_027M,S1501_C02_028E,S1501_C02_028M,S1501_C02_029E,S1501_C02_029M,S1501_C02_030E,S1501_C02_030M,S1501_C02_031E,S1501_C02_031M,S1501_C02_032E,S1501_C02_032M,S1501_C02_033E,S1501_C02_033M,S1501_C02_034E,S1501_C02_034M,S1501_C02_035E,S1501_C02_035M,S1501_C02_036E,S1501_C02_036M,S1501_C02_037E,S1501_C02_037M,S1501_C02_038E,S1501_C02_038M,S1501_C02_039E,S1501_C02_039M,S1501_C02_040E,S1501_C02_040M,S1501_C02_041E,S1501_C02_041M,S1501_C02_042E,S1501_C02_042M,S1501_C02_043E,S1501_C02_043M,S1501_C02_044E,S1501_C02_044M,S1501_C02_045E,S1501_C02_045M,S1501_C02_046E,S1501_C02_046M,S1501_C02_047E,S1501_C02_047M,S1501_C02_048E,S1501_C02_048M,S1501_C02_049E,S1501_C02_049M,S1501_C02_050E,S1501_C02_050M,S1501_C02_051E,S1501_C02_051M,S1501_C02_052E,S1501_C02_052M,S1501_C02_053E,S1501_C02_053M,S1501_C02_054E,S1501_C02_054M,S1501_C02_055E,S1501_C02_055M,S1501_C02_056E,S1501_C02_056M,S1501_C02_057E,S1501_C02_057M,S1501_C02_058E,S1501_C02_058M,S1501_C02_059E,S1501_C02_059M,S1501_C02_060E,S1501_C02_060M,S1501_C02_061E,S1501_C02_061M,S1501_C02_062E,S1501_C02_062M,S1501_C02_063E,S1501_C02_063M,S1501_C02_064E,S1501_C02_064M,S1501_C03_001E,S1501_C03_001M,S1501_C03_002E,S1501_C03_002M,S1501_C03_003E,S1501_C03_003M,S1501_C03_004E,S1501_C03_004M,S1501_C03_005E,S1501_C03_00

In [ ]:
education.shape

(3888, 770)

#### First Cleaning

In [ ]:
def first_clean(df):
  """
  Function cleans DataFrames from Census Bureau  to the essential information. From there on further cleaning has 
  to be done by hand (e.g. selecting columns)!
  """
  # Delete id column
  df.drop("GEO_ID", axis=1, inplace=True)

  # Set first row to column and delete first row
  df.columns = df.iloc[0, :]
  df.drop(0, inplace=True)
  
  # Rename "Geographic Area Name" to district column
  df.rename(columns={"Geographic Area Name": "district"}, inplace=True)

  # Delete all columns which show the margin of error
  margin_columns = [col for col in df.columns if "Margin" in col]
  df.drop(margin_columns, axis=1, inplace=True)

  # Change columnnames to lowercase
  df.columns = [col.lower() for col in df.columns]
  return df

In [ ]:
education = first_clean(education)
education.shape

(3887, 385)

#### DataFrame dependent cleaning

In [ ]:
def delete_male_female(df):
  df.drop(df.iloc[:, df.columns.str.contains("male|female")].columns, axis=1, inplace=True)
  return df

In [ ]:
education = delete_male_female(education)
education.shape

(3887, 129)

In [ ]:
# Just need columns with 18-24 and 25 or higher -> no more cohorts needed
education = education.iloc[:, :14]

In [ ]:
education.head()

,district,total!!estimate!!population 18 to 24 years,total!!estimate!!population 18 to 24 years!!less than high school graduate,total!!estimate!!population 18 to 24 years!!high school graduate (includes equivalency),total!!estimate!!population 18 to 24 years!!some college or associate's degree,total!!estimate!!population 18 to 24 years!!bachelor's degree or higher,total!!estimate!!population 25 years and over,total!!estimate!!population 25 years and over!!less than 9th grade,"total!!estimate!!population 25 years and over!!9th to 12th grade, no diploma",total!!estimate!!population 25 years and over!!high school graduate (includes equivalency),"total!!estimate!!population 25 years and over!!some college, no degree",total!!estimate!!population 25 years and over!!associate's degree,total!!estimate!!population 25 years and over!!bachelor's degree,total!!estimate!!population 25 years and over!!graduate or professional degree
1,"State House District 1 (2016), Alaska",2901,348,963,1351,239,11439,346,819,3632,3396,745,1545,956
2,"State House District 2 (2016), Alaska",3711,160,1685,1793,73,9253,223,265,2261,2894,932,1994,684
3,"State House District 3 (2016), Alaska",1929,260,582,979,108,11670,171,370,3186,3761,1356,2100,726
4,"State House District 4 (2016), Alaska",1906,281,547,737,341,13105,158,255,1559,3720,1057,3467,2889
5,"State House District 5 (2016), Alaska",2723,70,598,1834,221,11254,119,558,2078,3138,798,2628,1935


In [ ]:
less_than_highschool = [col for col in education.columns if "less than high school graduate" in col 
                        or "less than 9th grade" in col or "9th to 12th grade" in col]
highschool = [col for col in education if "high school graduate (includes equivalency)" in col]
college_associate = [col for col in education if "some college" in col or "associate's degree" in col]
bachelor_higher = [col for col in education if "bachelor's degree" in col or "graduate or professional degree" in col]

In [ ]:
less_than_highschool

['total!!estimate!!population 18 to 24 years!!less than high school graduate',
 'total!!estimate!!population 25 years and over!!less than 9th grade',
 'total!!estimate!!population 25 years and over!!9th to 12th grade, no diploma']

In [ ]:
education[education.columns[1:]] = education[education.columns[1:]].astype("int")

In [ ]:
education.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3887 entries, 1 to 3887
Data columns (total 14 columns):
 #   Column                                                                                      Non-Null Count  Dtype 
---  ------                                                                                      --------------  ----- 
 0   district                                                                                    3887 non-null   object
 1   total!!estimate!!population 18 to 24 years                                                  3887 non-null   int64 
 2   total!!estimate!!population 18 to 24 years!!less than high school graduate                  3887 non-null   int64 
 3   total!!estimate!!population 18 to 24 years!!high school graduate (includes equivalency)     3887 non-null   int64 
 4   total!!estimate!!population 18 to 24 years!!some college or associate's degree              3887 non-null   int64 
 5   total!!estimate!!population 18 to 24 years!!bach

In [ ]:
education["less than highschool"] = education[less_than_highschool].sum(axis=1)
education["highschool"] = education[highschool].sum(axis=1)
education["college or associate"] = education[college_associate].sum(axis=1)
education["bachelor or higher"] = education[bachelor_higher].sum(axis=1)

In [ ]:
education.head()

,district,total!!estimate!!population 18 to 24 years,total!!estimate!!population 18 to 24 years!!less than high school graduate,total!!estimate!!population 18 to 24 years!!high school graduate (includes equivalency),total!!estimate!!population 18 to 24 years!!some college or associate's degree,total!!estimate!!population 18 to 24 years!!bachelor's degree or higher,total!!estimate!!population 25 years and over,total!!estimate!!population 25 years and over!!less than 9th grade,"total!!estimate!!population 25 years and over!!9th to 12th grade, no diploma",total!!estimate!!population 25 years and over!!high school graduate (includes equivalency),"total!!estimate!!population 25 years and over!!some college, no degree",total!!estimate!!population 25 years and over!!associate's degree,total!!estimate!!population 25 years and over!!bachelor's degree,total!!estimate!!population 25 years and over!!graduate or professional degree,less than highschool,highschool,college or associate,bachelor or higher
1,"State House District 1 (2016), Alaska",2901,348,963,1351,239,11439,346,819,3632,3396,745,1545,956,1513,4595,5492,2740
2,"State House District 2 (2016), Alaska",3711,160,1685,1793,73,9253,223,265,2261,2894,932,1994,684,648,3946,5619,2751
3,"State House District 3 (2016), Alaska",1929,260,582,979,108,11670,171,370,3186,3761,1356,2100,726,801,3768,6096,2934
4,"State House District 4 (2016), Alaska",1906,281,547,737,341,13105,158,255,1559,3720,1057,3467,2889,694,2106,5514,6697
5,"State House District 5 (2016), Alaska",2723,70,598,1834,221,11254,119,558,2078,3138,798,2628,1935,747,2676,5770,4784


In [ ]:
education.shape

(3887, 18)

In [ ]:
# Delete columns
education = education.drop(education.columns[1:14], axis=1)
education.head()

,district,less than highschool,highschool,college or associate,bachelor or higher
1,"State House District 1 (2016), Alaska",1513,4595,5492,2740
2,"State House District 2 (2016), Alaska",648,3946,5619,2751
3,"State House District 3 (2016), Alaska",801,3768,6096,2934
4,"State House District 4 (2016), Alaska",694,2106,5514,6697
5,"State House District 5 (2016), Alaska",747,2676,5770,4784


#### Final Cleaning and District Dropping

In [ ]:
def clean_census_data(df, percentage=True):

  # Change strings to lower case
  df["district"] = df["district"].str.lower()

  # Get states out of district columns
  # 1. Replace ; in massachusetts with ,
  df.loc[df[df["district"] == "barnstable, dukes & nantucket district (2016); massachusetts"].index,
              "district"] = "barnstable, dukes & nantucket district (2016), massachusetts"
  # 2. Get states
  df["state"] = df["district"].apply(lambda x: x.split("(2016), ")[-1])

  
  # Clear district names

  # All state except Massachusetts and Vermont
  without_verm_mass = df[~df["district"].str.contains("massachusetts|vermont")]
  df.loc[without_verm_mass.index, "district"] = without_verm_mass["district"].apply(lambda x: x.split("district ")[1].split(" (2016)")[0])

  # Massachusetts
  massachusetts_district = df[df["state"] == "massachusetts"]["district"].apply(lambda x: x.split(" district")[0])
  df.loc[df[df["state"] == "massachusetts"].index, "district"] = massachusetts_district

  # Vermont
  vermont_district = df[df["state"] == "vermont"]["district"].apply(lambda x: x.split(" state")[0])
  df.loc[df[df["state"] == "vermont"].index, "district"] = vermont_district

  # Minor cleaning for South Carolina and Minnesota
  # South Carolina
  southcarolina_district = df[df["state"] == "south carolina"]["district"].apply(lambda x: x.split("-")[-1])
  southcarolina_district.iloc[:10] = southcarolina_district.iloc[:10].apply(lambda x: x.split("00")[-1])
  southcarolina_district[southcarolina_district.str.startswith("0")] = southcarolina_district[southcarolina_district.str.startswith("0")].str[1:]
  df.loc[df[df["state"] == "south carolina"].index, "district"] = southcarolina_district

  # Minnesota
  df.loc[df[df["state"] == "minnesota"].iloc[:18].index, "district"] = df[df["state"] == "minnesota"]["district"].iloc[:18].apply(lambda x: x[1:])

  # Change data types to numeric
  df.iloc[:, 1:-1] = df.iloc[:, 1:-1].astype("float")
  
  if percentage:
    # Calculate percentage values of features
    total_population = df.iloc[:, 1:-1].sum(axis=1)

    for col in df.iloc[:, 1:-1].columns:
        df[col] = df[col] / total_population
        df[col] = df[col].astype("float").round(4)
    
  return df

In [ ]:
education = clean_census_data(education)

In [ ]:
education.head()

,district,less than highschool,highschool,college or associate,bachelor or higher,state
1,1,0.1055,0.3204,0.3830,0.1911,alaska
2,2,0.0500,0.3044,0.4334,0.2122,alaska
3,3,0.0589,0.2771,0.4483,0.2158,alaska
4,4,0.0462,0.1403,0.3673,0.4461,alaska
5,5,0.0534,0.1915,0.4128,0.3423,alaska


In [ ]:
import pandas as pd
target_df = pd.read_csv("drive/MyDrive/US Elections/data_ballotpedia_2016.csv")
target_df.head()

,district,democrat,republican,other,state,year,target
0,1,1.0,0.0,0.0,alaska,2016,1
1,10,2021.0,5901.0,0.0,alaska,2016,0
2,11,0.0,5752.0,2681.0,alaska,2016,0
3,12,2061.0,5597.0,949.0,alaska,2016,0
4,13,0.0,1.0,0.0,alaska,2016,0


In [ ]:
def check_states_districts(target_df, check_df):
  """
  Function to check for missing states and districts in both the target and the check dataframe.
  """
  # Values which are in target df, but miss in new dataframe
  missing_in_check = []
  # Values which are in new dataframe, but miss in target df
  missing_in_target = []
  for state in target_df["state"].sort_values().unique():
    target_district = target_df[target_df["state"] == state]["district"]
    check_district = check_df[check_df["state"] == state]["district"]
    if state in check_df.state.values:
      #if (target_district.values == check_district.values).sum() == len(target_district.values):
      if sum(1 for a, b in zip(target_district.values, check_district.values) if a == b) == len(target_district.values):
        pass
      else:
        for district in target_district:
          if district not in check_district.values:
            missing_in_check.append([state, district])
        for district in check_district:
          if district not in target_district.values:
            missing_in_target.append([state, district])
  return missing_in_check, missing_in_target

In [ ]:
def drop_districts(df, missing_in_target):
  """
  Function to drop districts from the check_df which don't appear in the target_df. Works only for vermont and 
  west virginia. 
  """
  # Create lists for districts to delete to be saved in
  drop_vermont = []
  drop_west_virginia = []

  # Get all districts per state which have to be deleted
  for state, district in missing_in_target:
    if state == "vermont":
      drop_vermont.append(district)
    elif state == "west virginia":
      drop_west_virginia.append(district)

  # Delete districts
  df = df.drop(df[(df["state"] == "vermont") & (df["district"].isin(drop_vermont))].index)
  df = df.drop(df[(df["state"] == "west virginia") & (df["district"].isin(drop_west_virginia))].index)

  return df

In [ ]:
def drop_spare_districts(target_df, df):
  missing_in_check, missing_in_target = check_states_districts(target_df, df)
  if len(missing_in_check) > 0:
    print("Error: There should be no districts in target dataframe which are missing in the Census dataframe!")
  else:
    df = drop_districts(df, missing_in_target)
  return df

In [ ]:
education = drop_spare_districts(target_df, education)

In [ ]:
education.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3821 entries, 1 to 3887
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   district              3821 non-null   object 
 1   less than highschool  3821 non-null   float64
 2   highschool            3821 non-null   float64
 3   college or associate  3821 non-null   float64
 4   bachelor or higher    3821 non-null   float64
 5   state                 3821 non-null   object 
dtypes: float64(4), object(2)
memory usage: 209.0+ KB


#### Reset Index and Save DataFrame

In [ ]:
def reset_and_save(df, df_name:str):
  df = df.reset_index(drop=True)
  df.name = df_name
  df.to_csv("drive/MyDrive/US Elections/Census Data/Final Files/" + df.name + "_2016_final.csv", index=False)
  return df

In [ ]:
# Reset index so that it's possible to merge the dataframes on the indices
education = reset_and_save(education, "education_attainment")

## Occupation

The end goal is to have a dataframe which divides the occupations into different subcategories and shows for every district how many people work in which sector.

Steps:
1. Clean redundant columns✅
2. Divide occupations into categories✅
3. Include state column to better join them later with other dataframes✅
4. Clean district names so that they match the ones of my target variable dataframe✅
4. Calculate percentage values✅
5. Drop all districts which are not available in the target variable dataframe✅

In [ ]:
occupation = pd.read_csv("drive/MyDrive/US Elections/Census Data/Occupation 2016/occupation_2016.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,156,157,164,165,170,171,174,175,176,177,180,181,190,191,194,195,196,197,198,199,200,201,202,203,206,207,210,211,212,213,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,2

In [ ]:
occupation.head(3)

,GEO_ID,NAME,S2401_C01_001E,S2401_C01_001M,S2401_C01_002E,S2401_C01_002M,S2401_C01_003E,S2401_C01_003M,S2401_C01_004E,S2401_C01_004M,S2401_C01_005E,S2401_C01_005M,S2401_C01_006E,S2401_C01_006M,S2401_C01_007E,S2401_C01_007M,S2401_C01_008E,S2401_C01_008M,S2401_C01_009E,S2401_C01_009M,S2401_C01_010E,S2401_C01_010M,S2401_C01_011E,S2401_C01_011M,S2401_C01_012E,S2401_C01_012M,S2401_C01_013E,S2401_C01_013M,S2401_C01_014E,S2401_C01_014M,S2401_C01_015E,S2401_C01_015M,S2401_C01_016E,S2401_C01_016M,S2401_C01_017E,S2401_C01_017M,S2401_C01_018E,S2401_C01_018M,S2401_C01_019E,S2401_C01_019M,S2401_C01_020E,S2401_C01_020M,S2401_C01_021E,S2401_C01_021M,S2401_C01_022E,S2401_C01_022M,S2401_C01_023E,S2401_C01_023M,S2401_C01_024E,S2401_C01_024M,S2401_C01_025E,S2401_C01_025M,S2401_C01_026E,S2401_C01_026M,S2401_C01_027E,S2401_C01_027M,S2401_C01_028E,S2401_C01_028M,S2401_C01_029E,S2401_C01_029M,S2401_C01_030E,S2401_C01_030M,S2401_C01_031E,S2401_C01_031M,S2401_C01_032E,S2401_C01_032M,S2401_C01_033E,S2401_C01_033M,S2401_C01_034E,S2401_C01_034M,S2401_C01_035E,S2401_C01_035M,S2401_C01_036E,S2401_C01_036M,S2401_C02_001E,S2401_C02_001M,S2401_C02_002E,S2401_C02_002M,S2401_C02_003E,S2401_C02_003M,S2401_C02_004E,S2401_C02_004M,S2401_C02_005E,S2401_C02_005M,S2401_C02_006E,S2401_C02_006M,S2401_C02_007E,S2401_C02_007M,S2401_C02_008E,S2401_C02_008M,S2401_C02_009E,S2401_C02_009M,S2401_C02_010E,S2401_C02_010M,S2401_C02_011E,S2401_C02_011M,S2401_C02_012E,S2401_C02_012M,S2401_C02_013E,S2401_C02_013M,S2401_C02_014E,S2401_C02_014M,S2401_C02_015E,S2401_C02_015M,S2401_C02_016E,S2401_C02_016M,S2401_C02_017E,S2401_C02_017M,S2401_C02_018E,S2401_C02_018M,S2401_C02_019E,S2401_C02_019M,S2401_C02_020E,S2401_C02_020M,S2401_C02_021E,S2401_C02_021M,S2401_C02_022E,S2401_C02_022M,S2401_C02_023E,S2401_C02_023M,S2401_C02_024E,S2401_C02_024M,S2401_C02_025E,S2401_C02_025M,S2401_C02_026E,S2401_C02_026M,S2401_C02_027E,S2401_C02_027M,S2401_C02_028E,S2401_C02_028M,S2401_C02_029E,S2401_C02_029M,S2401_C02_030E,S2401_C02_030M,S2401_C02_031E,S2401_C02_031M,S2401_C02_032E,S2401_C02_032M,S2401_C02_033E,S2401_C02_033M,S2401_C02_034E,S2401_C02_034M,S2401_C02_035E,S2401_C02_035M,S2401_C02_036E,S2401_C02_036M,S2401_C03_001E,S2401_C03_001M,S2401_C03_002E,S2401_C03_002M,S2401_C03_003E,S2401_C03_003M,S2401_C03_004E,S2401_C03_004M,S2401_C03_005E,S2401_C03_005M,S2401_C03_006E,S2401_C03_006M,S2401_C03_007E,S2401_C03_007M,S2401_C03_008E,S2401_C03_008M,S2401_C03_009E,S2401_C03_009M,S2401_C03_010E,S2401_C03_010M,S2401_C03_011E,S2401_C03_011M,S2401_C03_012E,S2401_C03_012M,S2401_C03_013E,S2401_C03_013M,S2401_C03_014E,S2401_C03_014M,S2401_C03_015E,S2401_C03_015M,S2401_C03_016E,S2401_C03_016M,S2401_C03_017E,S2401_C03_017M,S2401_C03_018E,S2401_C03_018M,S2401_C03_019E,S2401_C03_019M,S2401_C03_020E,S2401_C03_020M,S2401_C03_021E,S2401_C03_021M,S2401_C03_022E,S2401_C03_022M,S2401_C03_023E,S2401_C03_023M,S2401_C03_024E,S2401_C03_024M,S2401_C03_025E,S2401_C03_025M,S2401_C03_026E,S2401_C03_026M,S2401_C03_027E,S2401_C03_027M,S2401_C03_028E,S2401_C03_028M,S2401_C03_029E,S2401_C03_029M,S2401_C03_030E,S2401_C03_030M,S2401_C03_031E,S2401_C03_031M,S2401_C03_032E,S2401_C03_032M,S2401_C03_033E,S2401_C03_033M,S2401_C03_034E,S2401_C03_034M,S2401_C03_035E,S2401_C03_035M,S2401_C03_036E,S2401_C03_036M,S2401_C04_001E,S2401_C04_001M,S2401_C04_002E,S2401_C04_002M,S2401_C04_003E,S2401_C04_003M,S2401_C04_004E,S2401_C04_004M,S2401_C04_005E,S2401_C04_005M,S2401_C04_006E,S2401_C04_006M,S2401_C04_007E,S2401_C04_007M,S2401_C04_008E,S2401_C04_008M,S2401_C04_009E,S2401_C04_009M,S2401_C04_010E,S2401_C04_010M,S2401_C04_011E,S2401_C04_011M,S2401_C04_012E,S2401_C04_012M,S2401_C04_013E,S2401_C04_013M,S2401_C04_014E,S2401_C04_014M,S2401_C04_015E,S2401_C04_015M,S2401_C04_016E,S2401_C04_016M,S2401_C04_017E,S2401_C04_017M,S2401_C04_018E,S2401_C04_018M,S2401_C04_019E,S2401_C04_019M,S2401_C04_020E,S2401_C04_020M,S2401_C04_021E,S2401_C04_021M,S2401_C04_022E,S2401_C04_022M,S2401_C04_023E,S2401_C04_023M,S2401_C04_024E,S2401_C04_024M,S2401_C04_025E,S2401_C04_02

#### First Cleaning

In [ ]:
occupation = first_clean(occupation)
occupation.shape

(3887, 181)

#### DataFrame dependent cleaning

In [ ]:
def delete_male_female(df):
  df.drop(df.iloc[:, df.columns.str.contains("Male|Female")].columns, axis=1, inplace=True)
  return df

In [ ]:
occupation = delete_male_female(occupation)
occupation.shape

(3887, 181)

In [ ]:
occupation.columns = occupation.columns.str.rsplit("total!!estimate!!").str[-1]

In [ ]:
occupation = occupation.iloc[:, occupation.columns.str.split("!!").str.len() == 1]

In [ ]:
occupation.head()

,district,civilian employed population 16 years and over,"management, business, science, and arts occupations",service occupations,sales and office occupations,"natural resources, construction, and maintenance occupations","production, transportation, and material moving occupations"
1,"State House District 1 (2016), Alaska",8496,2398,2005,2387,792,914
2,"State House District 2 (2016), Alaska",6308,2130,1319,1692,643,524
3,"State House District 3 (2016), Alaska",8918,3232,1072,1911,1649,1054
4,"State House District 4 (2016), Alaska",10875,4673,1932,1895,1450,925
5,"State House District 5 (2016), Alaska",9634,4157,1312,2320,960,885


In [ ]:
occupation.iloc[:, 1:] = occupation.iloc[:, 1:].apply(pd.to_numeric)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [ ]:
occupation["blue collar occupations"] = occupation.iloc[:,-2:].sum(axis=1)

In [ ]:
# Delete columns
to_delete = ["civilian employed population 16 years and over",
             "natural resources, construction, and maintenance occupations",
             "production, transportation, and material moving occupations"]
occupation.drop(to_delete, axis=1, inplace=True)

#### Final Cleaning and District Dropping

In [ ]:
occupation = clean_census_data(occupation)

In [ ]:
occupation.head()

,district,"management, business, science, and arts occupations",service occupations,sales and office occupations,blue collar occupations,state
1,1,0.2823,0.2360,0.2810,0.2008,alaska
2,2,0.3377,0.2091,0.2682,0.1850,alaska
3,3,0.3624,0.1202,0.2143,0.3031,alaska
4,4,0.4297,0.1777,0.1743,0.2184,alaska
5,5,0.4315,0.1362,0.2408,0.1915,alaska


In [ ]:
occupation = drop_spare_districts(target_df, occupation)

In [ ]:
occupation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3821 entries, 1 to 3887
Data columns (total 6 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   district                                             3821 non-null   object 
 1   management, business, science, and arts occupations  3821 non-null   float64
 2   service occupations                                  3821 non-null   float64
 3   sales and office occupations                         3821 non-null   float64
 4   blue collar occupations                              3821 non-null   float64
 5   state                                                3821 non-null   object 
dtypes: float64(4), object(2)
memory usage: 209.0+ KB


#### Reset Index and Save DataFrame

In [ ]:
# Reset index so that it's possible to merge the dataframes on the indices
occupation = reset_and_save(occupation, "occupation")
occupation.tail()

,district,"management, business, science, and arts occupations",service occupations,sales and office occupations,blue collar occupations,state
3816,56,0.3146,0.2057,0.2139,0.2658,wyoming
3817,57,0.3132,0.1385,0.2364,0.3119,wyoming
3818,58,0.2616,0.1842,0.2198,0.3344,wyoming
3819,59,0.2086,0.2188,0.2511,0.3215,wyoming
3820,60,0.2477,0.1878,0.2385,0.3260,wyoming


## Earnings

In [ ]:
earnings = pd.read_csv("/content/drive/MyDrive/US Elections/Census Data/Earnings 2016/earnings_2016.csv")

In [ ]:
earnings.head(1)

,GEO_ID,NAME,S2001_C01_001E,S2001_C01_001M,S2001_C01_002E,S2001_C01_002M,S2001_C01_003E,S2001_C01_003M,S2001_C01_004E,S2001_C01_004M,S2001_C01_005E,S2001_C01_005M,S2001_C01_006E,S2001_C01_006M,S2001_C01_007E,S2001_C01_007M,S2001_C01_008E,S2001_C01_008M,S2001_C01_009E,S2001_C01_009M,S2001_C01_010E,S2001_C01_010M,S2001_C01_011E,S2001_C01_011M,S2001_C01_012E,S2001_C01_012M,S2001_C01_013E,S2001_C01_013M,S2001_C01_014E,S2001_C01_014M,S2001_C01_015E,S2001_C01_015M,S2001_C01_016E,S2001_C01_016M,S2001_C01_017E,S2001_C01_017M,S2001_C01_018E,S2001_C01_018M,S2001_C01_019E,S2001_C01_019M,S2001_C01_020E,S2001_C01_020M,S2001_C01_021E,S2001_C01_021M,S2001_C02_001E,S2001_C02_001M,S2001_C02_002E,S2001_C02_002M,S2001_C02_003E,S2001_C02_003M,S2001_C02_004E,S2001_C02_004M,S2001_C02_005E,S2001_C02_005M,S2001_C02_006E,S2001_C02_006M,S2001_C02_007E,S2001_C02_007M,S2001_C02_008E,S2001_C02_008M,S2001_C02_009E,S2001_C02_009M,S2001_C02_010E,S2001_C02_010M,S2001_C02_011E,S2001_C02_011M,S2001_C02_012E,S2001_C02_012M,S2001_C02_013E,S2001_C02_013M,S2001_C02_014E,S2001_C02_014M,S2001_C02_015E,S2001_C02_015M,S2001_C02_016E,S2001_C02_016M,S2001_C02_017E,S2001_C02_017M,S2001_C02_018E,S2001_C02_018M,S2001_C02_019E,S2001_C02_019M,S2001_C02_020E,S2001_C02_020M,S2001_C02_021E,S2001_C02_021M,S2001_C03_001E,S2001_C03_001M,S2001_C03_002E,S2001_C03_002M,S2001_C03_003E,S2001_C03_003M,S2001_C03_004E,S2001_C03_004M,S2001_C03_005E,S2001_C03_005M,S2001_C03_006E,S2001_C03_006M,S2001_C03_007E,S2001_C03_007M,S2001_C03_008E,S2001_C03_008M,S2001_C03_009E,S2001_C03_009M,S2001_C03_010E,S2001_C03_010M,S2001_C03_011E,S2001_C03_011M,S2001_C03_012E,S2001_C03_012M,S2001_C03_013E,S2001_C03_013M,S2001_C03_014E,S2001_C03_014M,S2001_C03_015E,S2001_C03_015M,S2001_C03_016E,S2001_C03_016M,S2001_C03_017E,S2001_C03_017M,S2001_C03_018E,S2001_C03_018M,S2001_C03_019E,S2001_C03_019M,S2001_C03_020E,S2001_C03_020M,S2001_C03_021E,S2001_C03_021M
0,id,Geographic Area Name,Total!!Estimate!!Population 16 years and over ...,Total!!Margin of Error!!Population 16 years an...,Total!!Estimate!!Median earnings (dollars),Total!!Margin of Error!!Median earnings (dollars),"Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...","Total!!Estimate!!Full-time, year-round workers...","Total!!Margin of Error!!Full-time, year-round ...",Total!!Estimate!!Median earnings (dollars),Total!!Margin of Error!!Median earnings (dollars),Total!!Estimate!!Mean earnings (dollars),Total!!Margin of Error!!Mean earnings (dollars),Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!MEDIAN EARNINGS BY EDUCATIONA...,Total!!Margin of Error!!MEDIAN EARNINGS BY EDU...,Total!!Estimate!!PERCENT ALLOCATED!!Earnings i...,Total!

#### First Cleaning

In [ ]:
earnings = first_clean(earnings)

In [ ]:
earnings = delete_male_female(earnings)
earnings.head(1)

,district,total!!estimate!!population 16 years and over with earnings,total!!estimate!!median earnings (dollars),"total!!estimate!!full-time, year-round workers with earnings","total!!estimate!!full-time, year-round workers with earnings!!$1 to $9,999 or loss","total!!estimate!!full-time, year-round workers with earnings!!$10,000 to $14,999","total!!estimate!!full-time, year-round workers with earnings!!$15,000 to $24,999","total!!estimate!!full-time, year-round workers with earnings!!$25,000 to $34,999","total!!estimate!!full-time, year-round workers with earnings!!$35,000 to $49,999","total!!estimate!!full-time, year-round workers with earnings!!$50,000 to $64,999","total!!estimate!!full-time, year-round workers with earnings!!$65,000 to $74,999","total!!estimate!!full-time, year-round workers with earnings!!$75,000 to $99,999","total!!estimate!!full-time, year-round workers with earnings!!$100,000 or more",total!!estimate!!median earnings (dollars),total!!estimate!!mean earnings (dollars),total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings,total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!less than high school graduate,total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!high school graduate (includes equivalency),total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!some college or associate's degree,total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!bachelor's degree,total!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!graduate or professional degree,total!!estimate!!percent allocated!!earnings in the past 12 months,male!!estimate!!population 16 years and over with earnings,male!!estimate!!median earnings (dollars),"male!!estimate!!full-time, year-round workers with earnings","male!!estimate!!full-time, year-round workers with earnings!!$1 to $9,999 or loss","male!!estimate!!full-time, year-round workers with earnings!!$10,000 to $14,999","male!!estimate!!full-time, year-round workers with earnings!!$15,000 to $24,999","male!!estimate!!full-time, year-round workers with earnings!!$25,000 to $34,999","male!!estimate!!full-time, year-round workers with earnings!!$35,000 to $49,999","male!!estimate!!full-time, year-round workers with earnings!!$50,000 to $64,999","male!!estimate!!full-time, year-round workers with earnings!!$65,000 to $74,999","male!!estimate!!full-time, year-round workers with earnings!!$75,000 to $99,999","male!!estimate!!full-time, year-round workers with earnings!!$100,000 or more",male!!estimate!!median earnings (dollars),male!!estimate!!mean earnings (dollars),male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings,male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!less than high school graduate,male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!high school graduate (includes equivalency),male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!some college or associate's degree,male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!bachelor's degree,male!!estimate!!median earnings by educational attainment!!population 25 years and over with earnings!!graduate or professional degree,male!!estimate!!percent allocated!!earnings in the past 12 months,female!!estimate!!population 16 years and over with earnings,female!!estimate!!median earnings (dollars),"female!!estimate!!full-time, year-round workers with earnings","female!!estimate!!full-time, year-round workers with earnings!!$1 to $9,999 or loss","female!!estimate!!full-time, year-round workers with earnings

In [ ]:
earnings.columns = earnings.columns.str.rsplit("total!!estimate!!").str[-1]

#### DataFrame dependent cleaning

In [ ]:
# Save separately, because of cleaning functions
median_earnings = earnings.iloc[:, 2]

# Delete unnecessary columns
earnings = pd.concat([earnings.iloc[:, 0], earnings.iloc[:, 4:13]], axis=1)

earnings.head(1)

,district,"full-time, year-round workers with earnings!!$1 to $9,999 or loss","full-time, year-round workers with earnings!!$10,000 to $14,999","full-time, year-round workers with earnings!!$15,000 to $24,999","full-time, year-round workers with earnings!!$25,000 to $34,999","full-time, year-round workers with earnings!!$35,000 to $49,999","full-time, year-round workers with earnings!!$50,000 to $64,999","full-time, year-round workers with earnings!!$65,000 to $74,999","full-time, year-round workers with earnings!!$75,000 to $99,999","full-time, year-round workers with earnings!!$100,000 or more"
1,"State House District 1 (2016), Alaska",1.2,1.1,15.6,16.8,23.9,15.7,8.1,8.9,8.6


In [ ]:
# Change Data Type to float
earnings.iloc[:, 1:] = earnings.iloc[:, 1:].astype("float")

In [ ]:
# Make new categories of earnings
earnings["earnings $1 - $24'999"] = earnings.iloc[:, 1:4].sum(axis=1)
earnings["earnings $25'000 - 49'999"] = earnings.iloc[:, 4:6].sum(axis=1)
earnings["earnings $50'000 - 74'999"] = earnings.iloc[:, 6:8].sum(axis=1)
earnings["earnings $75'000 or more"] = earnings.iloc[:, 8:10].sum(axis=1)

In [ ]:
final_columns = [col for col in earnings.columns if "full-time" not in col]
earnings = earnings[final_columns]
earnings.head(1)

,district,earnings $1 - $24'999,earnings $25'000 - 49'999,earnings $50'000 - 74'999,earnings $75'000 or more
1,"State House District 1 (2016), Alaska",17.9,40.7,23.8,17.5


#### Final Cleaning and District Dropping

In [ ]:
earnings = clean_census_data(earnings)
earnings.head(1)

,district,earnings $1 - $24'999,earnings $25'000 - 49'999,earnings $50'000 - 74'999,earnings $75'000 or more,state
1,1,0.1792,0.4074,0.2382,0.1752,alaska


In [ ]:
earnings.shape

(3887, 6)

In [ ]:
earnings = drop_spare_districts(target_df, earnings)
earnings.shape

(3821, 6)

In [ ]:
earnings["median earnings"] = median_earnings
earnings.head(1)

,district,earnings $1 - $24'999,earnings $25'000 - 49'999,earnings $50'000 - 74'999,earnings $75'000 or more,state,median earnings
1,1,0.1792,0.4074,0.2382,0.1752,alaska,30518


#### Reset Index and Save DataFrame

In [ ]:
df = reset_and_save(earnings, "earnings")
df.tail()

,district,earnings $1 - $24'999,earnings $25'000 - 49'999,earnings $50'000 - 74'999,earnings $75'000 or more,state,median earnings
3816,56,0.1588,0.3656,0.2537,0.2218,wyoming,30719
3817,57,0.2560,0.3840,0.2270,0.1330,wyoming,29402
3818,58,0.2124,0.4098,0.2214,0.1563,wyoming,30601
3819,59,0.2713,0.3954,0.1902,0.1431,wyoming,25100
3820,60,0.1270,0.2940,0.2680,0.3110,wyoming,34050


## Poverty

In [ ]:
poverty = pd.read_csv("drive/MyDrive/US Elections/Census Data/Poverty 2016/poverty_2016.csv")
poverty.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,286,2

,GEO_ID,NAME,S1701_C01_001E,S1701_C01_001M,S1701_C01_002E,S1701_C01_002M,S1701_C01_003E,S1701_C01_003M,S1701_C01_004E,S1701_C01_004M,S1701_C01_005E,S1701_C01_005M,S1701_C01_006E,S1701_C01_006M,S1701_C01_007E,S1701_C01_007M,S1701_C01_008E,S1701_C01_008M,S1701_C01_009E,S1701_C01_009M,S1701_C01_010E,S1701_C01_010M,S1701_C01_011E,S1701_C01_011M,S1701_C01_012E,S1701_C01_012M,S1701_C01_013E,S1701_C01_013M,S1701_C01_014E,S1701_C01_014M,S1701_C01_015E,S1701_C01_015M,S1701_C01_016E,S1701_C01_016M,S1701_C01_017E,S1701_C01_017M,S1701_C01_018E,S1701_C01_018M,S1701_C01_019E,S1701_C01_019M,S1701_C01_020E,S1701_C01_020M,S1701_C01_021E,S1701_C01_021M,S1701_C01_022E,S1701_C01_022M,S1701_C01_023E,S1701_C01_023M,S1701_C01_024E,S1701_C01_024M,S1701_C01_025E,S1701_C01_025M,S1701_C01_026E,S1701_C01_026M,S1701_C01_027E,S1701_C01_027M,S1701_C01_028E,S1701_C01_028M,S1701_C01_029E,S1701_C01_029M,S1701_C01_030E,S1701_C01_030M,S1701_C01_031E,S1701_C01_031M,S1701_C01_032E,S1701_C01_032M,S1701_C01_033E,S1701_C01_033M,S1701_C01_034E,S1701_C01_034M,S1701_C01_035E,S1701_C01_035M,S1701_C01_036E,S1701_C01_036M,S1701_C01_037E,S1701_C01_037M,S1701_C01_038E,S1701_C01_038M,S1701_C01_039E,S1701_C01_039M,S1701_C01_040E,S1701_C01_040M,S1701_C01_041E,S1701_C01_041M,S1701_C01_042E,S1701_C01_042M,S1701_C01_043E,S1701_C01_043M,S1701_C01_044E,S1701_C01_044M,S1701_C01_045E,S1701_C01_045M,S1701_C01_046E,S1701_C01_046M,S1701_C01_047E,S1701_C01_047M,S1701_C01_048E,S1701_C01_048M,S1701_C01_049E,S1701_C01_049M,S1701_C01_050E,S1701_C01_050M,S1701_C01_051E,S1701_C01_051M,S1701_C01_052E,S1701_C01_052M,S1701_C01_053E,S1701_C01_053M,S1701_C01_054E,S1701_C01_054M,S1701_C01_055E,S1701_C01_055M,S1701_C01_056E,S1701_C01_056M,S1701_C01_057E,S1701_C01_057M,S1701_C01_058E,S1701_C01_058M,S1701_C01_059E,S1701_C01_059M,S1701_C01_060E,S1701_C01_060M,S1701_C01_061E,S1701_C01_061M,S1701_C02_001E,S1701_C02_001M,S1701_C02_002E,S1701_C02_002M,S1701_C02_003E,S1701_C02_003M,S1701_C02_004E,S1701_C02_004M,S1701_C02_005E,S1701_C02_005M,S1701_C02_006E,S1701_C02_006M,S1701_C02_007E,S1701_C02_007M,S1701_C02_008E,S1701_C02_008M,S1701_C02_009E,S1701_C02_009M,S1701_C02_010E,S1701_C02_010M,S1701_C02_011E,S1701_C02_011M,S1701_C02_012E,S1701_C02_012M,S1701_C02_013E,S1701_C02_013M,S1701_C02_014E,S1701_C02_014M,S1701_C02_015E,S1701_C02_015M,S1701_C02_016E,S1701_C02_016M,S1701_C02_017E,S1701_C02_017M,S1701_C02_018E,S1701_C02_018M,S1701_C02_019E,S1701_C02_019M,S1701_C02_020E,S1701_C02_020M,S1701_C02_021E,S1701_C02_021M,S1701_C02_022E,S1701_C02_022M,S1701_C02_023E,S1701_C02_023M,S1701_C02_024E,S1701_C02_024M,S1701_C02_025E,S1701_C02_025M,S1701_C02_026E,S1701_C02_026M,S1701_C02_027E,S1701_C02_027M,S1701_C02_028E,S1701_C02_028M,S1701_C02_029E,S1701_C02_029M,S1701_C02_030E,S1701_C02_030M,S1701_C02_031E,S1701_C02_031M,S1701_C02_032E,S1701_C02_032M,S1701_C02_033E,S1701_C02_033M,S1701_C02_034E,S1701_C02_034M,S1701_C02_035E,S1701_C02_035M,S1701_C02_036E,S1701_C02_036M,S1701_C02_037E,S1701_C02_037M,S1701_C02_038E,S1701_C02_038M,S1701_C02_039E,S1701_C02_039M,S1701_C02_040E,S1701_C02_040M,S1701_C02_041E,S1701_C02_041M,S1701_C02_042E,S1701_C02_042M,S1701_C02_043E,S1701_C02_043M,S1701_C02_044E,S1701_C02_044M,S1701_C02_045E,S1701_C02_045M,S1701_C02_046E,S1701_C02_046M,S1701_C02_047E,S1701_C02_047M,S1701_C02_048E,S1701_C02_048M,S1701_C02_049E,S1701_C02_049M,S1701_C02_050E,S1701_C02_050M,S1701_C02_051E,S1701_C02_051M,S1701_C02_052E,S1701_C02_052M,S1701_C02_053E,S1701_C02_053M,S1701_C02_054E,S1701_C02_054M,S1701_C02_055E,S1701_C02_055M,S1701_C02_056E,S1701_C02_056M,S1701_C02_057E,S1701_C02_057M,S1701_C02_058E,S1701_C02_058M,S1701_C02_059E,S1701_C02_059M,S1701_C02_060E,S1701_C02_060M,S1701_C02_061E,S1701_C02_061M,S1701_C03_001E,S1701_C03_001M,S1701_C03_002E,S1701_C03_002M,S1701_C03_003E,S1701_C03_003M,S1701_C03_004E,S1701_C03_004M,S1701_C03_005E,S1701_C03_005M,S1701_C03_006E,S1701_C03_006M,S1701_C03_007E,S1701_C03_007M,S1701_C03_008E,S1701_C03_008M,S1701_C03_009E,S1701_C03_009M,S1701_C03_010E,S1701_C03_010M,S1701_C03_011E,S1701_C03_01

In [ ]:
poverty.shape

(3888, 368)

#### First Cleaning

In [ ]:
poverty = first_clean(poverty)
poverty.shape

(3887, 184)

In [ ]:
poverty.head(1)

,district,total!!estimate!!population for whom poverty status is determined,total!!estimate!!age!!under 18 years,total!!estimate!!age!!under 18 years!!under 5 years,total!!estimate!!age!!under 18 years!!5 to 17 years,total!!estimate!!age!!under 18 years!!related children of householder under 18 years,total!!estimate!!age!!18 to 64 years,total!!estimate!!age!!18 to 64 years!!18 to 34 years,total!!estimate!!age!!18 to 64 years!!35 to 64 years,total!!estimate!!age!!60 years and over,total!!estimate!!age!!65 years and over,total!!estimate!!sex!!male,total!!estimate!!sex!!female,total!!estimate!!race and hispanic or latino origin!!white alone,total!!estimate!!race and hispanic or latino origin!!black or african american alone,total!!estimate!!race and hispanic or latino origin!!american indian and alaska native alone,total!!estimate!!race and hispanic or latino origin!!asian alone,total!!estimate!!race and hispanic or latino origin!!native hawaiian and other pacific islander alone,total!!estimate!!race and hispanic or latino origin!!some other race alone,total!!estimate!!race and hispanic or latino origin!!two or more races,total!!estimate!!hispanic or latino origin (of any race),"total!!estimate!!white alone, not hispanic or latino",total!!estimate!!educational attainment!!population 25 years and over,total!!estimate!!educational attainment!!population 25 years and over!!less than high school graduate,total!!estimate!!educational attainment!!population 25 years and over!!high school graduate (includes equivalency),"total!!estimate!!educational attainment!!population 25 years and over!!some college, associate's degree",total!!estimate!!educational attainment!!population 25 years and over!!bachelor's degree or higher,total!!estimate!!employment status!!civilian labor force 16 years and over,total!!estimate!!employment status!!civilian labor force 16 years and over!!employed,total!!estimate!!employment status!!civilian labor force 16 years and over!!employed!!male,total!!estimate!!employment status!!civilian labor force 16 years and over!!employed!!female,total!!estimate!!employment status!!civilian labor force 16 years and over!!unemployed,total!!estimate!!employment status!!civilian labor force 16 years and over!!unemployed!!male,total!!estimate!!employment status!!civilian labor force 16 years and over!!unemployed!!female,total!!estimate!!work experience!!population 16 years and over,"total!!estimate!!work experience!!population 16 years and over!!worked full-time, year-round in the past 12 months",total!!estimate!!work experience!!population 16 years and over!!worked part-time or part-year in the past 12 months,total!!estimate!!work experience!!population 16 years and over!!did not work,total!!estimate!!all individuals with income below the following poverty ratios!!50 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!125 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!150 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!185 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!200 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!300 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!400 percent of poverty level,total!!estimate!!all individuals with income below the following poverty ratios!!500 percent of poverty level,total!!estimate!!unrelated individuals for whom poverty status is determined,total!!estimate!!unrelated individuals for whom poverty status is determined!!male,total!!estimate!!unrelated individuals for whom poverty status is determined!!female,total!!estimate!!15 years,total!!estimate!!16 to 17 years,total!!estimate!!18 to 24 years,total!!estimate!!25 to 34 years,total!!estimate!!35 t

#### DataFrame dependent cleaning

In [ ]:
percent_columns = [col for col in poverty.columns if "percent below poverty level" in col or col == "district"]
poverty = poverty[percent_columns].iloc[:, :2]
poverty.rename(columns={"percent below poverty level!!estimate!!population for whom poverty status is determined": "percent of tot. population in poverty"},
               inplace=True)
poverty.head()

,district,percent of tot. population in poverty
1,"State House District 1 (2016), Alaska",15.3
2,"State House District 2 (2016), Alaska",6.4
3,"State House District 3 (2016), Alaska",7.4
4,"State House District 4 (2016), Alaska",5.1
5,"State House District 5 (2016), Alaska",5.9


#### Final Cleaning and District dropping

In [ ]:
poverty = clean_census_data(poverty, percentage=False)
poverty.head()

,district,percent of tot. population in poverty,state
1,1,15.3,alaska
2,2,6.4,alaska
3,3,7.4,alaska
4,4,5.1,alaska
5,5,5.9,alaska


In [ ]:
poverty = drop_spare_districts(target_df, poverty)
poverty.shape

(3821, 3)

#### Reset Index and Save DataFrame

In [ ]:
poverty = reset_and_save(poverty, "poverty")
poverty.tail()

,district,percent of tot. population in poverty,state
3816,56,11.7,wyoming
3817,57,12.2,wyoming
3818,58,11.3,wyoming
3819,59,16.6,wyoming
3820,60,9.4,wyoming


## Foreign Borns

Percentage of foreign born on total population in district.

In [ ]:
import pandas as pd
foreign_borns = pd.read_csv("drive/MyDrive/US Elections/Census Data/Foreign Born 2016/foreign_born_2016.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,60,61,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,168,169,170,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,196,197,200,201,204,205,206,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,262,263,266,267,268,269,270,272,273,274,275,276,277,278,280,281,282,283,284,285,286,288,289,290,291,292,293,294,296,297,298,299,300,301,3

In [ ]:
foreign_borns.head(1)

,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,DP02_0002PM,DP02_0003E,DP02_0003M,DP02_0003PE,DP02_0003PM,DP02_0004E,DP02_0004M,DP02_0004PE,DP02_0004PM,DP02_0005E,DP02_0005M,DP02_0005PE,DP02_0005PM,DP02_0006E,DP02_0006M,DP02_0006PE,DP02_0006PM,DP02_0007E,DP02_0007M,DP02_0007PE,DP02_0007PM,DP02_0008E,DP02_0008M,DP02_0008PE,DP02_0008PM,DP02_0009E,DP02_0009M,DP02_0009PE,DP02_0009PM,DP02_0010E,DP02_0010M,DP02_0010PE,DP02_0010PM,DP02_0011E,DP02_0011M,DP02_0011PE,DP02_0011PM,DP02_0012E,DP02_0012M,DP02_0012PE,DP02_0012PM,DP02_0013E,DP02_0013M,DP02_0013PE,DP02_0013PM,DP02_0014E,DP02_0014M,DP02_0014PE,DP02_0014PM,DP02_0015E,DP02_0015M,DP02_0015PE,DP02_0015PM,DP02_0016E,DP02_0016M,DP02_0016PE,DP02_0016PM,DP02_0017E,DP02_0017M,DP02_0017PE,DP02_0017PM,DP02_0018E,DP02_0018M,DP02_0018PE,DP02_0018PM,DP02_0019E,DP02_0019M,DP02_0019PE,DP02_0019PM,DP02_0020E,DP02_0020M,DP02_0020PE,DP02_0020PM,DP02_0021E,DP02_0021M,DP02_0021PE,DP02_0021PM,DP02_0022E,DP02_0022M,DP02_0022PE,DP02_0022PM,DP02_0023E,DP02_0023M,DP02_0023PE,DP02_0023PM,DP02_0024E,DP02_0024M,DP02_0024PE,DP02_0024PM,DP02_0025E,DP02_0025M,DP02_0025PE,DP02_0025PM,DP02_0026E,DP02_0026M,DP02_0026PE,DP02_0026PM,DP02_0027E,DP02_0027M,DP02_0027PE,DP02_0027PM,DP02_0028E,DP02_0028M,DP02_0028PE,DP02_0028PM,DP02_0029E,DP02_0029M,DP02_0029PE,DP02_0029PM,DP02_0030E,DP02_0030M,DP02_0030PE,DP02_0030PM,DP02_0031E,DP02_0031M,DP02_0031PE,DP02_0031PM,DP02_0032E,DP02_0032M,DP02_0032PE,DP02_0032PM,DP02_0033E,DP02_0033M,DP02_0033PE,DP02_0033PM,DP02_0034E,DP02_0034M,DP02_0034PE,DP02_0034PM,DP02_0035E,DP02_0035M,DP02_0035PE,DP02_0035PM,DP02_0036E,DP02_0036M,DP02_0036PE,DP02_0036PM,DP02_0037E,DP02_0037M,DP02_0037PE,DP02_0037PM,DP02_0038E,DP02_0038M,DP02_0038PE,DP02_0038PM,DP02_0039E,DP02_0039M,DP02_0039PE,DP02_0039PM,DP02_0040E,DP02_0040M,DP02_0040PE,DP02_0040PM,DP02_0041E,DP02_0041M,DP02_0041PE,DP02_0041PM,DP02_0042E,DP02_0042M,DP02_0042PE,DP02_0042PM,DP02_0043E,DP02_0043M,DP02_0043PE,DP02_0043PM,DP02_0044E,DP02_0044M,DP02_0044PE,DP02_0044PM,DP02_0045E,DP02_0045M,DP02_0045PE,DP02_0045PM,DP02_0046E,DP02_0046M,DP02_0046PE,DP02_0046PM,DP02_0047E,DP02_0047M,DP02_0047PE,DP02_0047PM,DP02_0048E,DP02_0048M,DP02_0048PE,DP02_0048PM,DP02_0049E,DP02_0049M,DP02_0049PE,DP02_0049PM,DP02_0050E,DP02_0050M,DP02_0050PE,DP02_0050PM,DP02_0051E,DP02_0051M,DP02_0051PE,DP02_0051PM,DP02_0052E,DP02_0052M,DP02_0052PE,DP02_0052PM,DP02_0053E,DP02_0053M,DP02_0053PE,DP02_0053PM,DP02_0054E,DP02_0054M,DP02_0054PE,DP02_0054PM,DP02_0055E,DP02_0055M,DP02_0055PE,DP02_0055PM,DP02_0056E,DP02_0056M,DP02_0056PE,DP02_0056PM,DP02_0057E,DP02_0057M,DP02_0057PE,DP02_0057PM,DP02_0058E,DP02_0058M,DP02_0058PE,DP02_0058PM,DP02_0059E,DP02_0059M,DP02_0059PE,DP02_0059PM,DP02_0060E,DP02_0060M,DP02_0060PE,DP02_0060PM,DP02_0061E,DP02_0061M,DP02_0061PE,DP02_0061PM,DP02_0062E,DP02_0062M,DP02_0062PE,DP02_0062PM,DP02_0063E,DP02_0063M,DP02_0063PE,DP02_0063PM,DP02_0064E,DP02_0064M,DP02_0064PE,DP02_0064PM,DP02_0065E,DP02_0065M,DP02_0065PE,DP02_0065PM,DP02_0066E,DP02_0066M,DP02_0066PE,DP02_0066PM,DP02_0067E,DP02_0067M,DP02_0067PE,DP02_0067PM,DP02_0068E,DP02_0068M,DP02_0068PE,DP02_0068PM,DP02_0069E,DP02_0069M,DP02_0069PE,DP02_0069PM,DP02_0070E,DP02_0070M,DP02_0070PE,DP02_0070PM,DP02_0071E,DP02_0071M,DP02_0071PE,DP02_0071PM,DP02_0072E,DP02_0072M,DP02_0072PE,DP02_0072PM,DP02_0073E,DP02_0073M,DP02_0073PE,DP02_0073PM,DP02_0074E,DP02_0074M,DP02_0074PE,DP02_0074PM,DP02_0075E,DP02_0075M,DP02_0075PE,DP02_0075PM,DP02_0076E,DP02_0076M,DP02_0076PE,DP02_0076PM,DP02_0077E,DP02_0077M,DP02_0077PE,DP02_0077PM,DP02_0078E,DP02_0078M,DP02_0078PE,DP02_0078PM,DP02_0079E,DP02_0079M,DP02_0079PE,DP02_0079PM,DP02_0080E,DP02_0080M,DP02_0080PE,DP02_0080PM,DP02_0081E,DP02_0081M,DP02_0081PE,DP02_0081PM,DP02_0082E,DP02_0082M,DP02_0082PE,DP02_0082PM,DP02_0083E,DP02_0083M,DP02_0083PE,DP02_0083PM,DP02_0084E,DP02_0084M,DP02_0084PE,DP02_0084PM,DP02_0085E,DP02_0085M,DP02_0085PE,DP02_0085PM,DP02_0086E,DP02_0086M,DP02_0086PE,DP02_0086PM,DP02_0087E,DP02_0087M,DP02_0087PE,DP02_0087

In [ ]:
foreign_borns.shape

(3888, 610)

#### First Cleaning

In [ ]:
foreign_borns = first_clean(foreign_borns)
foreign_borns.head(1)

,estimate!!households by type!!total households,percent!!households by type!!total households,estimate!!households by type!!total households!!family households (families),percent!!households by type!!total households!!family households (families),estimate!!households by type!!total households!!family households (families)!!with own children of the householder under 18 years,percent!!households by type!!total households!!family households (families)!!with own children of the householder under 18 years,estimate!!households by type!!total households!!family households (families)!!married-couple family,percent!!households by type!!total households!!family households (families)!!married-couple family,estimate!!households by type!!total households!!family households (families)!!married-couple family!!with own children of the householder under 18 years,percent!!households by type!!total households!!family households (families)!!married-couple family!!with own children of the householder under 18 years,"estimate!!households by type!!total households!!family households (families)!!male householder, no wife present, family","percent!!households by type!!total households!!family households (families)!!male householder, no wife present, family","estimate!!households by type!!total households!!family households (families)!!male householder, no wife present, family!!with own children of the householder under 18 years","percent!!households by type!!total households!!family households (families)!!male householder, no wife present, family!!with own children of the householder under 18 years","estimate!!households by type!!total households!!family households (families)!!female householder, no husband present, family","percent!!households by type!!total households!!family households (families)!!female householder, no husband present, family","estimate!!households by type!!total households!!family households (families)!!female householder, no husband present, family!!with own children of the householder under 18 years","percent!!households by type!!total households!!family households (families)!!female householder, no husband present, family!!with own children of the householder under 18 years",estimate!!households by type!!total households!!nonfamily households,percent!!households by type!!total households!!nonfamily households,estimate!!households by type!!total households!!nonfamily households!!householder living alone,percent!!households by type!!total households!!nonfamily households!!householder living alone,estimate!!households by type!!total households!!nonfamily households!!householder living alone!!65 years and over,percent!!households by type!!total households!!nonfamily households!!householder living alone!!65 years and over,estimate!!households by type!!households with one or more people under 18 years,percent!!households by type!!households with one or more people under 18 years,estimate!!households by type!!households with one or more people 65 years and over,percent!!households by type!!households with one or more people 65 years and over,estimate!!households by type!!average household size,percent!!households by type!!average household size,estimate!!households by type!!average family size,percent!!households by type!!average family size,estimate!!relationship!!population in households,percent!!relationship!!population in households,estimate!!relationship!!population in households!!householder,percent!!relationship!!population in households!!householder,estimate!!relationship!!population in households!!spouse,percent!!relationship!!population in households!!spouse,estimate!!relationship!!population in households!!child,percent!!relationship!!population in households!!child,estimate!!relationship!!population in households!!other relatives,percent!!relationship!!population in households!!other relatives,estimate!!relationship!!population in households!!nonrelatives,percent!!relationship!!population in households!!nonrelatives,estim

#### DataFrame dependent Cleaning

In [ ]:
# Delete all percent columns
estimate_cols = [col for col in foreign_borns.columns if "percent" not in col]
foreign_borns = foreign_borns[estimate_cols]

# Select only Place of Birth columns
citizenship_cols = [col for col in foreign_borns.columns if "place of birth" in col or col == "district"]
foreign_borns = foreign_borns[citizenship_cols]
foreign_borns.head(1)

,estimate!!place of birth!!total population,estimate!!place of birth!!total population!!native,estimate!!place of birth!!total population!!native!!born in united states,estimate!!place of birth!!total population!!native!!born in united states!!state of residence,estimate!!place of birth!!total population!!native!!born in united states!!different state,"estimate!!place of birth!!total population!!native!!born in puerto rico, u.s. island areas, or born abroad to american parent(s)",estimate!!place of birth!!total population!!foreign born,district
1,18129,16578,15955,7080,8875,623,1551,"State House District 1 (2016), Alaska"


In [ ]:
# Select columns
district = foreign_borns.district
foreign_borns = foreign_borns.drop(foreign_borns.iloc[:, [0, 2, 3, 4, 5, 7]].columns, axis=1)
foreign_borns.head()

,estimate!!place of birth!!total population!!native,estimate!!place of birth!!total population!!foreign born
1,16578,1551
2,17193,1205
3,18279,552
4,18739,272
5,16997,1075


In [ ]:
# Insert district on index 0 to be able to clean dataframe with clean_census_data function
foreign_borns.insert(loc=0, column='district', value=district)

In [ ]:
foreign_borns.columns = foreign_borns.columns.str.split("!!").str[-1]

#### Final Cleaning and Ditrict dropping

In [ ]:
foreign_borns = clean_census_data(foreign_borns)

In [ ]:
foreign_borns.head()

,district,native,foreign born,state
1,1,0.9144,0.0856,alaska
2,2,0.9345,0.0655,alaska
3,3,0.9707,0.0293,alaska
4,4,0.9857,0.0143,alaska
5,5,0.9405,0.0595,alaska


In [ ]:
foreign_borns = drop_spare_districts(target_df, foreign_borns)
foreign_borns.shape

(3821, 4)

#### Reset Index and Save DataFrame

In [ ]:
foreign_borns = reset_and_save(foreign_borns, "foreign_borns")

In [ ]:
foreign_borns.tail()

,district,native,foreign born,state
3816,56,0.9660,0.0340,wyoming
3817,57,0.9749,0.0251,wyoming
3818,58,0.9853,0.0147,wyoming
3819,59,0.9623,0.0377,wyoming
3820,60,0.9432,0.0568,wyoming


## Average Family Size

In [ ]:
family_size = pd.read_csv("/content/drive/MyDrive/US Elections/Census Data/Family Size & Housing Tenure 2016/familiy_size_housing_tenure_2016.csv")

In [ ]:
family_size.head(1)

,GEO_ID,NAME,S1101_C01_001E,S1101_C01_001M,S1101_C01_002E,S1101_C01_002M,S1101_C01_003E,S1101_C01_003M,S1101_C01_004E,S1101_C01_004M,S1101_C01_005E,S1101_C01_005M,S1101_C01_006E,S1101_C01_006M,S1101_C01_007E,S1101_C01_007M,S1101_C01_008E,S1101_C01_008M,S1101_C01_009E,S1101_C01_009M,S1101_C01_010E,S1101_C01_010M,S1101_C01_011E,S1101_C01_011M,S1101_C01_012E,S1101_C01_012M,S1101_C01_013E,S1101_C01_013M,S1101_C01_014E,S1101_C01_014M,S1101_C01_015E,S1101_C01_015M,S1101_C01_016E,S1101_C01_016M,S1101_C01_017E,S1101_C01_017M,S1101_C01_018E,S1101_C01_018M,S1101_C01_019E,S1101_C01_019M,S1101_C01_020E,S1101_C01_020M,S1101_C02_001E,S1101_C02_001M,S1101_C02_002E,S1101_C02_002M,S1101_C02_003E,S1101_C02_003M,S1101_C02_004E,S1101_C02_004M,S1101_C02_005E,S1101_C02_005M,S1101_C02_006E,S1101_C02_006M,S1101_C02_007E,S1101_C02_007M,S1101_C02_008E,S1101_C02_008M,S1101_C02_009E,S1101_C02_009M,S1101_C02_010E,S1101_C02_010M,S1101_C02_011E,S1101_C02_011M,S1101_C02_012E,S1101_C02_012M,S1101_C02_013E,S1101_C02_013M,S1101_C02_014E,S1101_C02_014M,S1101_C02_015E,S1101_C02_015M,S1101_C02_016E,S1101_C02_016M,S1101_C02_017E,S1101_C02_017M,S1101_C02_018E,S1101_C02_018M,S1101_C02_019E,S1101_C02_019M,S1101_C02_020E,S1101_C02_020M,S1101_C03_001E,S1101_C03_001M,S1101_C03_002E,S1101_C03_002M,S1101_C03_003E,S1101_C03_003M,S1101_C03_004E,S1101_C03_004M,S1101_C03_005E,S1101_C03_005M,S1101_C03_006E,S1101_C03_006M,S1101_C03_007E,S1101_C03_007M,S1101_C03_008E,S1101_C03_008M,S1101_C03_009E,S1101_C03_009M,S1101_C03_010E,S1101_C03_010M,S1101_C03_011E,S1101_C03_011M,S1101_C03_012E,S1101_C03_012M,S1101_C03_013E,S1101_C03_013M,S1101_C03_014E,S1101_C03_014M,S1101_C03_015E,S1101_C03_015M,S1101_C03_016E,S1101_C03_016M,S1101_C03_017E,S1101_C03_017M,S1101_C03_018E,S1101_C03_018M,S1101_C03_019E,S1101_C03_019M,S1101_C03_020E,S1101_C03_020M,S1101_C04_001E,S1101_C04_001M,S1101_C04_002E,S1101_C04_002M,S1101_C04_003E,S1101_C04_003M,S1101_C04_004E,S1101_C04_004M,S1101_C04_005E,S1101_C04_005M,S1101_C04_006E,S1101_C04_006M,S1101_C04_007E,S1101_C04_007M,S1101_C04_008E,S1101_C04_008M,S1101_C04_009E,S1101_C04_009M,S1101_C04_010E,S1101_C04_010M,S1101_C04_011E,S1101_C04_011M,S1101_C04_012E,S1101_C04_012M,S1101_C04_013E,S1101_C04_013M,S1101_C04_014E,S1101_C04_014M,S1101_C04_015E,S1101_C04_015M,S1101_C04_016E,S1101_C04_016M,S1101_C04_017E,S1101_C04_017M,S1101_C04_018E,S1101_C04_018M,S1101_C04_019E,S1101_C04_019M,S1101_C04_020E,S1101_C04_020M,S1101_C05_001E,S1101_C05_001M,S1101_C05_002E,S1101_C05_002M,S1101_C05_003E,S1101_C05_003M,S1101_C05_004E,S1101_C05_004M,S1101_C05_005E,S1101_C05_005M,S1101_C05_006E,S1101_C05_006M,S1101_C05_007E,S1101_C05_007M,S1101_C05_008E,S1101_C05_008M,S1101_C05_009E,S1101_C05_009M,S1101_C05_010E,S1101_C05_010M,S1101_C05_011E,S1101_C05_011M,S1101_C05_012E,S1101_C05_012M,S1101_C05_013E,S1101_C05_013M,S1101_C05_014E,S1101_C05_014M,S1101_C05_015E,S1101_C05_015M,S1101_C05_016E,S1101_C05_016M,S1101_C05_017E,S1101_C05_017M,S1101_C05_018E,S1101_C05_018M,S1101_C05_019E,S1101_C05_019M,S1101_C05_020E,S1101_C05_020M
0,id,Geographic Area Name,Total!!Estimate!!Total households,Total!!Margin of Error!!Total households,Total!!Estimate!!Average household size,Total!!Margin of Error!!Average household size,Total!!Estimate!!FAMILIES!!Total families,Total!!Margin of Error!!FAMILIES!!Total families,Total!!Estimate!!FAMILIES!!Average family size,Total!!Margin of Error!!FAMILIES!!Average fami...,Total!!Estimate!!AGE OF OWN CHILDREN!!Househol...,Total!!Margin of Error!!AGE OF OWN CHILDREN!!H...,Total!!Estimate!!AGE OF OWN CHILDREN!!Househol...,Total!!Margin of Error!!AGE OF OWN CHILDREN!!H...,Total!!Estimate!!AGE OF OWN CHILDREN!!Househol...,Total!!Margin of Error!!AGE OF OWN CHILDREN!!H...,Total!!Estimate!!AGE OF OWN CHILDREN!!Househol...,Total!!Margin of Error!!AGE OF OWN CHILDREN!!H...,Total!!Estimate!!Total households,Total!!Margin of Error!!Total households,Total!!Estimate!!Total households!!SELECTED HO...,Total!!Margin of Error!!Total households!!SELE...,Total!!Estimate!!Total households!!S

In [ ]:
family_size.shape

(3888, 202)

#### First Cleaning

In [ ]:
family_size = first_clean(family_size)

In [ ]:
family_size.head(1)

,district,total!!estimate!!total households,total!!estimate!!average household size,total!!estimate!!families!!total families,total!!estimate!!families!!average family size,total!!estimate!!age of own children!!households with own children of the householder under 18 years,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years only,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years and 6 to 17 years,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!6 to 17 years only,total!!estimate!!total households,total!!estimate!!total households!!selected households by type!!households with one or more people under 18 years,total!!estimate!!total households!!selected households by type!!households with one or more people 60 years and over,total!!estimate!!total households!!selected households by type!!householder living alone,total!!estimate!!total households!!selected households by type!!householder living alone!!65 years and over,total!!estimate!!unmarried-partner households!!same sex,total!!estimate!!unmarried-partner households!!opposite sex,total!!estimate!!units in structure!!1-unit structures,total!!estimate!!units in structure!!2-or-more-unit structures,total!!estimate!!units in structure!!mobile homes and all other types of units,total!!estimate!!housing tenure!!owner-occupied housing units,total!!estimate!!housing tenure!!renter-occupied housing units,married-couple family household!!estimate!!total households,married-couple family household!!estimate!!average household size,married-couple family household!!estimate!!families!!total families,married-couple family household!!estimate!!families!!average family size,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years only,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years and 6 to 17 years,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!6 to 17 years only,married-couple family household!!estimate!!total households,married-couple family household!!estimate!!total households!!selected households by type!!households with one or more people under 18 years,married-couple family household!!estimate!!total households!!selected households by type!!households with one or more people 60 years and over,married-couple family household!!estimate!!total households!!selected households by type!!householder living alone,married-couple family household!!estimate!!total households!!selected households by type!!householder living alone!!65 years and over,married-couple family household!!estimate!!unmarried-partner households!!same sex,married-couple family household!!estimate!!unmarried-partner households!!opposite sex,married-couple family household!!estimate!!units in structure!!1-unit structures,married-couple family household!!estimate!!units in structure!!2-or-more-unit structures,married-couple family household!!estimate!!units in structure!!mobile homes and all other types of units,married-couple family household!!estimate!!housing tenure!!owner-occupied housing units,married-couple family household!!estimate!!housing tenure!!renter-occupied housing units,"male householder, no wife present, family household!!estimate!!total households","male householder, no wife present, family household!!estimate!!average household size","male householder, no wife present, family household!!estimate!!families!!total families","male householder, no wife present, family household!!estimate!!families!!average family size","male householder, no wife present,

#### DataFrame dependent Cleaning

In [ ]:
columns = [col for col in family_size.columns if "average family size" in col or col == "district"]
family_size = family_size[columns]
family_size.head()

,district,total!!estimate!!families!!average family size,married-couple family household!!estimate!!families!!average family size,"male householder, no wife present, family household!!estimate!!families!!average family size","female householder, no husband present, family household!!estimate!!families!!average family size",nonfamily household!!estimate!!families!!average family size
1,"State House District 1 (2016), Alaska",3.34,3.34,3.07,3.46,(X)
2,"State House District 2 (2016), Alaska",3.43,3.45,2.90,3.53,(X)
3,"State House District 3 (2016), Alaska",3.48,3.41,3.10,4.04,(X)
4,"State House District 4 (2016), Alaska",3.19,3.15,3.67,3.42,(X)
5,"State House District 5 (2016), Alaska",3.46,3.43,3.38,3.75,(X)


In [ ]:
family_size = family_size.drop(family_size.iloc[:, 2:].columns, axis=1)
family_size.head()

,district,total!!estimate!!families!!average family size
1,"State House District 1 (2016), Alaska",3.34
2,"State House District 2 (2016), Alaska",3.43
3,"State House District 3 (2016), Alaska",3.48
4,"State House District 4 (2016), Alaska",3.19
5,"State House District 5 (2016), Alaska",3.46


In [ ]:
family_size.columns = ["district", family_size.columns[1].split("!!")[-1]]

#### Final Cleaning and Ditrict dropping

In [ ]:
family_size = clean_census_data(family_size, percentage=False)
family_size.head()

,district,average family size,state
1,1,3.34,alaska
2,2,3.43,alaska
3,3,3.48,alaska
4,4,3.19,alaska
5,5,3.46,alaska


In [ ]:
family_size = drop_spare_districts(target_df, family_size)
family_size.shape

(3821, 3)

#### Reset Index and Save DataFrame

In [ ]:
family_size = reset_and_save(family_size, "family_size")
family_size.tail()

,district,average family size,state
3816,56,2.89,wyoming
3817,57,3.25,wyoming
3818,58,2.86,wyoming
3819,59,3.09,wyoming
3820,60,3.25,wyoming


## Housing Tenure

Percentage of owner-occupied housing units per district. -> vs. renter-occupied housing units

In [ ]:
housing_tenure = pd.read_csv("/content/drive/MyDrive/US Elections/Census Data/Family Size & Housing Tenure 2016/familiy_size_housing_tenure_2016.csv")

#### First Cleaning

In [ ]:
housing_tenure = first_clean(housing_tenure)
housing_tenure.head(1)

,district,total!!estimate!!total households,total!!estimate!!average household size,total!!estimate!!families!!total families,total!!estimate!!families!!average family size,total!!estimate!!age of own children!!households with own children of the householder under 18 years,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years only,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years and 6 to 17 years,total!!estimate!!age of own children!!households with own children of the householder under 18 years!!6 to 17 years only,total!!estimate!!total households,total!!estimate!!total households!!selected households by type!!households with one or more people under 18 years,total!!estimate!!total households!!selected households by type!!households with one or more people 60 years and over,total!!estimate!!total households!!selected households by type!!householder living alone,total!!estimate!!total households!!selected households by type!!householder living alone!!65 years and over,total!!estimate!!unmarried-partner households!!same sex,total!!estimate!!unmarried-partner households!!opposite sex,total!!estimate!!units in structure!!1-unit structures,total!!estimate!!units in structure!!2-or-more-unit structures,total!!estimate!!units in structure!!mobile homes and all other types of units,total!!estimate!!housing tenure!!owner-occupied housing units,total!!estimate!!housing tenure!!renter-occupied housing units,married-couple family household!!estimate!!total households,married-couple family household!!estimate!!average household size,married-couple family household!!estimate!!families!!total families,married-couple family household!!estimate!!families!!average family size,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years only,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!under 6 years and 6 to 17 years,married-couple family household!!estimate!!age of own children!!households with own children of the householder under 18 years!!6 to 17 years only,married-couple family household!!estimate!!total households,married-couple family household!!estimate!!total households!!selected households by type!!households with one or more people under 18 years,married-couple family household!!estimate!!total households!!selected households by type!!households with one or more people 60 years and over,married-couple family household!!estimate!!total households!!selected households by type!!householder living alone,married-couple family household!!estimate!!total households!!selected households by type!!householder living alone!!65 years and over,married-couple family household!!estimate!!unmarried-partner households!!same sex,married-couple family household!!estimate!!unmarried-partner households!!opposite sex,married-couple family household!!estimate!!units in structure!!1-unit structures,married-couple family household!!estimate!!units in structure!!2-or-more-unit structures,married-couple family household!!estimate!!units in structure!!mobile homes and all other types of units,married-couple family household!!estimate!!housing tenure!!owner-occupied housing units,married-couple family household!!estimate!!housing tenure!!renter-occupied housing units,"male householder, no wife present, family household!!estimate!!total households","male householder, no wife present, family household!!estimate!!average household size","male householder, no wife present, family household!!estimate!!families!!total families","male householder, no wife present, family household!!estimate!!families!!average family size","male householder, no wife present,

In [ ]:
housing_tenure.shape

(3887, 101)

#### DataFrame dependent Cleaning

In [ ]:
columns = [col for col in housing_tenure.columns if "housing tenure" in col or col == "district"]
housing_tenure = housing_tenure[columns]
housing_tenure.head(1)

,district,total!!estimate!!housing tenure!!owner-occupied housing units,total!!estimate!!housing tenure!!renter-occupied housing units,married-couple family household!!estimate!!housing tenure!!owner-occupied housing units,married-couple family household!!estimate!!housing tenure!!renter-occupied housing units,"male householder, no wife present, family household!!estimate!!housing tenure!!owner-occupied housing units","male householder, no wife present, family household!!estimate!!housing tenure!!renter-occupied housing units","female householder, no husband present, family household!!estimate!!housing tenure!!owner-occupied housing units","female householder, no husband present, family household!!estimate!!housing tenure!!renter-occupied housing units",nonfamily household!!estimate!!housing tenure!!owner-occupied housing units,nonfamily household!!estimate!!housing tenure!!renter-occupied housing units
1,"State House District 1 (2016), Alaska",42.3,57.7,64.0,36.0,28.3,71.7,36.3,63.7,30.6,69.4


In [ ]:
housing_tenure = housing_tenure.drop(housing_tenure.iloc[:, 2:], axis=1)
housing_tenure.head(1)

,district,total!!estimate!!housing tenure!!owner-occupied housing units
1,"State House District 1 (2016), Alaska",42.3


In [ ]:
housing_tenure.columns = ["district", "percent " + housing_tenure.columns[1].split("!!")[-1]]

#### Final Cleaning and Ditrict dropping

In [ ]:
housing_tenure = clean_census_data(housing_tenure, percentage=False)

In [ ]:
housing_tenure = drop_spare_districts(target_df, housing_tenure)
housing_tenure.shape

(3821, 3)

#### Reset Index and Save DataFrame

In [ ]:
housing_tenure = reset_and_save(housing_tenure, "housing_tenure")
housing_tenure.tail()

,district,percent owner-occupied housing units,state
3816,56,55.5,wyoming
3817,57,57.6,wyoming
3818,58,71.5,wyoming
3819,59,55.6,wyoming
3820,60,75.1,wyoming


## Monthly Housing Costs

In [ ]:
housing_costs = pd.read_csv("drive/MyDrive/US Elections/Census Data/Monthly Housing Costs 2016/housing_costs_2016.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,2

In [ ]:
housing_costs.shape

(3888, 278)

#### First Cleaning

In [ ]:
housing_costs = first_clean(housing_costs)
housing_costs.head(1)

,district,occupied housing units!!estimate!!occupied housing units,"occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!less than $5,000","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$5,000 to $9,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$10,000 to $14,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$15,000 to $19,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$20,000 to $24,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$25,000 to $34,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$35,000 to $49,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$50,000 to $74,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$75,000 to $99,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$100,000 to $149,999","occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!$150,000 or more",occupied housing units!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!median household income (dollars),occupied housing units!!estimate!!monthly housing costs!!less than $300,occupied housing units!!estimate!!monthly housing costs!!$300 to $499,occupied housing units!!estimate!!monthly housing costs!!$500 to $799,occupied housing units!!estimate!!monthly housing costs!!$800 to $999,"occupied housing units!!estimate!!monthly housing costs!!$1,000 to $1,499","occupied housing units!!estimate!!monthly housing costs!!$1,500 to $1,999","occupied housing units!!estimate!!monthly housing costs!!$2,000 to $2,499","occupied housing units!!estimate!!monthly housing costs!!$2,500 to $2,999","occupied housing units!!estimate!!monthly housing costs!!$3,000 or more",occupied housing units!!estimate!!monthly housing costs!!no cash rent,occupied housing units!!estimate!!monthly housing costs!!median (dollars),"occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!less than $20,000","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!less than $20,000!!less than 20 percent","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!less than $20,000!!20 to 29 percent","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!less than $20,000!!30 percent or more","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$20,000 to $34,999","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$20,000 to $34,999!!less than 20 percent","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$20,000 to $34,999!!20 to 29 percent","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$20,000 to $34,999!!30 percent or more","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$35,000 to $49,999","occupied housing units!!estimate!!monthly housing costs as a percentage of household income in the past 12 months!!$35,000 to $49,999!!less than 20 percent","occupied housing units!!estimate!!mon

#### DataFrame dependent Cleaning

In [ ]:
# Select total housings (not owner or rented housings)
occupied_housing_columns = [col for col in housing_costs.columns if col.startswith("occupied housing units") or
                            col == "district"]
housing_costs = housing_costs[occupied_housing_columns]

# Delete start of the column names which are the same everywhere
housing_costs.columns = housing_costs.columns.str.split("estimate!!").str[-1]

# Select only columns which show monthly housing costs of all housings independent of who lives in these houses
monthly_costs_columns = [col for col in housing_costs.columns if col.startswith("monthly housing costs!!") or
                         col == "district"]
housing_costs = housing_costs[monthly_costs_columns]
housing_costs.head(1)

,district,monthly housing costs!!less than $300,monthly housing costs!!$300 to $499,monthly housing costs!!$500 to $799,monthly housing costs!!$800 to $999,"monthly housing costs!!$1,000 to $1,499","monthly housing costs!!$1,500 to $1,999","monthly housing costs!!$2,000 to $2,499","monthly housing costs!!$2,500 to $2,999","monthly housing costs!!$3,000 or more",monthly housing costs!!no cash rent,monthly housing costs!!median (dollars)
1,"State House District 1 (2016), Alaska",3.7,5.9,11.5,15.4,31.7,14.6,10.4,2.6,1.7,2.5,1165


In [ ]:
housing_costs.iloc[:, 1:] = housing_costs.iloc[:, 1:].astype("float")

In [ ]:
housing_costs["monthly housing costs - $499 or lower"] = housing_costs.iloc[:, 1:3].sum(axis=1)
housing_costs["monthly housing costs - $500 - 999"] = housing_costs.iloc[:, 3:5].sum(axis=1)
housing_costs["monthly housing costs - $1000 1999"] = housing_costs.iloc[:, 5:7].sum(axis=1)
housing_costs["monthly housing costs - $2000 or higher"] = housing_costs.iloc[:, 7:10].sum(axis=1)
housing_costs["monthly housing costs - median"] = housing_costs.iloc[:, 11]
housing_costs.drop(housing_costs.iloc[:, 1:12], axis=1, inplace=True)
housing_costs.head()

,district,monthly housing costs - $499 or lower,monthly housing costs - $500 - 999,monthly housing costs - $1000 1999,monthly housing costs - $2000 or higher,monthly housing costs - median
1,"State House District 1 (2016), Alaska",9.6,26.9,46.3,14.7,1165.0
2,"State House District 2 (2016), Alaska",4.5,15.4,40.6,35.0,1674.0
3,"State House District 3 (2016), Alaska",7.2,17.6,43.7,30.9,1680.0
4,"State House District 4 (2016), Alaska",9.2,25.2,40.2,22.5,1328.0
5,"State House District 5 (2016), Alaska",10.0,25.0,44.7,20.0,1242.0


#### Final Cleaning and Ditrict dropping

In [ ]:
housing_costs = clean_census_data(housing_costs, percentage=False)

In [ ]:
housing_costs = drop_spare_districts(target_df, housing_costs)
housing_costs.shape

(3821, 7)

#### Reset Index and Save DataFrame

In [ ]:
housing_costs = reset_and_save(housing_costs, "housing_costs")
housing_costs.tail()

,district,monthly housing costs - $499 or lower,monthly housing costs - $500 - 999,monthly housing costs - $1000 1999,monthly housing costs - $2000 or higher,monthly housing costs - median,state
3816,56,25.8,36.6,28.0,6.7,722.0,wyoming
3817,57,21.8,38.7,36.8,2.8,895.0,wyoming
3818,58,31.2,33.3,28.2,4.4,734.0,wyoming
3819,59,27.3,38.8,28.9,1.8,753.0,wyoming
3820,60,28.5,24.5,41.1,5.1,935.0,wyoming


## Age

* age = percentage of population in the respective cohorts and median age of district population


In [ ]:
age_sex = pd.read_csv("drive/MyDrive/US Elections/Census Data/Age 2016/age_2016.csv")

In [ ]:
age_sex.shape

(3888, 218)

#### First Cleaning

In [ ]:
age_sex = first_clean(age_sex)
age_sex.head(1)

,district,total!!estimate!!total population,total!!estimate!!age!!under 5 years,total!!estimate!!age!!5 to 9 years,total!!estimate!!age!!10 to 14 years,total!!estimate!!age!!15 to 19 years,total!!estimate!!age!!20 to 24 years,total!!estimate!!age!!25 to 29 years,total!!estimate!!age!!30 to 34 years,total!!estimate!!age!!35 to 39 years,total!!estimate!!age!!40 to 44 years,total!!estimate!!age!!45 to 49 years,total!!estimate!!age!!50 to 54 years,total!!estimate!!age!!55 to 59 years,total!!estimate!!age!!60 to 64 years,total!!estimate!!age!!65 to 69 years,total!!estimate!!age!!70 to 74 years,total!!estimate!!age!!75 to 79 years,total!!estimate!!age!!80 to 84 years,total!!estimate!!age!!85 years and over,total!!estimate!!selected age categories!!5 to 14 years,total!!estimate!!selected age categories!!15 to 17 years,total!!estimate!!selected age categories!!18 to 24 years,total!!estimate!!selected age categories!!15 to 44 years,total!!estimate!!selected age categories!!16 years and over,total!!estimate!!selected age categories!!18 years and over,total!!estimate!!selected age categories!!60 years and over,total!!estimate!!selected age categories!!62 years and over,total!!estimate!!selected age categories!!65 years and over,total!!estimate!!selected age categories!!75 years and over,total!!estimate!!summary indicators!!median age (years),total!!estimate!!summary indicators!!sex ratio (males per 100 females),total!!estimate!!summary indicators!!age dependency ratio,total!!estimate!!summary indicators!!old-age dependency ratio,total!!estimate!!summary indicators!!child dependency ratio,total!!estimate!!percent allocated!!sex,total!!estimate!!percent allocated!!age,male!!estimate!!total population,male!!estimate!!age!!under 5 years,male!!estimate!!age!!5 to 9 years,male!!estimate!!age!!10 to 14 years,male!!estimate!!age!!15 to 19 years,male!!estimate!!age!!20 to 24 years,male!!estimate!!age!!25 to 29 years,male!!estimate!!age!!30 to 34 years,male!!estimate!!age!!35 to 39 years,male!!estimate!!age!!40 to 44 years,male!!estimate!!age!!45 to 49 years,male!!estimate!!age!!50 to 54 years,male!!estimate!!age!!55 to 59 years,male!!estimate!!age!!60 to 64 years,male!!estimate!!age!!65 to 69 years,male!!estimate!!age!!70 to 74 years,male!!estimate!!age!!75 to 79 years,male!!estimate!!age!!80 to 84 years,male!!estimate!!age!!85 years and over,male!!estimate!!selected age categories!!5 to 14 years,male!!estimate!!selected age categories!!15 to 17 years,male!!estimate!!selected age categories!!18 to 24 years,male!!estimate!!selected age categories!!15 to 44 years,male!!estimate!!selected age categories!!16 years and over,male!!estimate!!selected age categories!!18 years and over,male!!estimate!!selected age categories!!60 years and over,male!!estimate!!selected age categories!!62 years and over,male!!estimate!!selected age categories!!65 years and over,male!!estimate!!selected age categories!!75 years and over,male!!estimate!!summary indicators!!median age (years),male!!estimate!!summary indicators!!sex ratio (males per 100 females),male!!estimate!!summary indicators!!age dependency ratio,male!!estimate!!summary indicators!!old-age dependency ratio,male!!estimate!!summary indicators!!child dependency ratio,male!!estimate!!percent allocated!!sex,male!!estimate!!percent allocated!!age,female!!estimate!!total population,female!!estimate!!age!!under 5 years,female!!estimate!!age!!5 to 9 years,female!!estimate!!age!!10 to 14 years,female!!estimate!!age!!15 to 19 years,female!!estimate!!age!!20 to 24 years,female!!estimate!!age!!25 to 29 years,female!!estimate!!age!!30 to 34 years,female!!estimate!!age!!35 to 39 years,female!!estimate!!age!!40 to 44 years,female!!estimate!!age!!45 to 49 years,female!!estimate!!age!!50 to 54 years,female!!estimate!!age!!55 to 59 years,female!!estimate!!age!!60 to 64 years,female!!estimate!!age!!65 to 69 years,female!!estimate!!age!!70 to 74 years,female!!estimate!!age!!75 to 79 years,female!!estimate!!age!!80 to 84 years,

In [ ]:
age_sex.columns

Index(['district', 'total!!estimate!!total population',
       'total!!estimate!!age!!under 5 years',
       'total!!estimate!!age!!5 to 9 years',
       'total!!estimate!!age!!10 to 14 years',
       'total!!estimate!!age!!15 to 19 years',
       'total!!estimate!!age!!20 to 24 years',
       'total!!estimate!!age!!25 to 29 years',
       'total!!estimate!!age!!30 to 34 years',
       'total!!estimate!!age!!35 to 39 years',
       ...
       'female!!estimate!!selected age categories!!62 years and over',
       'female!!estimate!!selected age categories!!65 years and over',
       'female!!estimate!!selected age categories!!75 years and over',
       'female!!estimate!!summary indicators!!median age (years)',
       'female!!estimate!!summary indicators!!sex ratio (males per 100 females)',
       'female!!estimate!!summary indicators!!age dependency ratio',
       'female!!estimate!!summary indicators!!old-age dependency ratio',
       'female!!estimate!!summary indicators!!child depe

#### DataFrame dependent Cleaning

In [ ]:
age = age_sex.copy()
age = delete_male_female(age)
age.head(1)

,district,total!!estimate!!total population,total!!estimate!!age!!under 5 years,total!!estimate!!age!!5 to 9 years,total!!estimate!!age!!10 to 14 years,total!!estimate!!age!!15 to 19 years,total!!estimate!!age!!20 to 24 years,total!!estimate!!age!!25 to 29 years,total!!estimate!!age!!30 to 34 years,total!!estimate!!age!!35 to 39 years,total!!estimate!!age!!40 to 44 years,total!!estimate!!age!!45 to 49 years,total!!estimate!!age!!50 to 54 years,total!!estimate!!age!!55 to 59 years,total!!estimate!!age!!60 to 64 years,total!!estimate!!age!!65 to 69 years,total!!estimate!!age!!70 to 74 years,total!!estimate!!age!!75 to 79 years,total!!estimate!!age!!80 to 84 years,total!!estimate!!age!!85 years and over,total!!estimate!!selected age categories!!5 to 14 years,total!!estimate!!selected age categories!!15 to 17 years,total!!estimate!!selected age categories!!18 to 24 years,total!!estimate!!selected age categories!!15 to 44 years,total!!estimate!!selected age categories!!16 years and over,total!!estimate!!selected age categories!!18 years and over,total!!estimate!!selected age categories!!60 years and over,total!!estimate!!selected age categories!!62 years and over,total!!estimate!!selected age categories!!65 years and over,total!!estimate!!selected age categories!!75 years and over,total!!estimate!!summary indicators!!median age (years),total!!estimate!!summary indicators!!sex ratio (males per 100 females),total!!estimate!!summary indicators!!age dependency ratio,total!!estimate!!summary indicators!!old-age dependency ratio,total!!estimate!!summary indicators!!child dependency ratio,total!!estimate!!percent allocated!!sex,total!!estimate!!percent allocated!!age,male!!estimate!!total population,male!!estimate!!age!!under 5 years,male!!estimate!!age!!5 to 9 years,male!!estimate!!age!!10 to 14 years,male!!estimate!!age!!15 to 19 years,male!!estimate!!age!!20 to 24 years,male!!estimate!!age!!25 to 29 years,male!!estimate!!age!!30 to 34 years,male!!estimate!!age!!35 to 39 years,male!!estimate!!age!!40 to 44 years,male!!estimate!!age!!45 to 49 years,male!!estimate!!age!!50 to 54 years,male!!estimate!!age!!55 to 59 years,male!!estimate!!age!!60 to 64 years,male!!estimate!!age!!65 to 69 years,male!!estimate!!age!!70 to 74 years,male!!estimate!!age!!75 to 79 years,male!!estimate!!age!!80 to 84 years,male!!estimate!!age!!85 years and over,male!!estimate!!selected age categories!!5 to 14 years,male!!estimate!!selected age categories!!15 to 17 years,male!!estimate!!selected age categories!!18 to 24 years,male!!estimate!!selected age categories!!15 to 44 years,male!!estimate!!selected age categories!!16 years and over,male!!estimate!!selected age categories!!18 years and over,male!!estimate!!selected age categories!!60 years and over,male!!estimate!!selected age categories!!62 years and over,male!!estimate!!selected age categories!!65 years and over,male!!estimate!!selected age categories!!75 years and over,male!!estimate!!summary indicators!!median age (years),male!!estimate!!summary indicators!!sex ratio (males per 100 females),male!!estimate!!summary indicators!!age dependency ratio,male!!estimate!!summary indicators!!old-age dependency ratio,male!!estimate!!summary indicators!!child dependency ratio,male!!estimate!!percent allocated!!sex,male!!estimate!!percent allocated!!age,female!!estimate!!total population,female!!estimate!!age!!under 5 years,female!!estimate!!age!!5 to 9 years,female!!estimate!!age!!10 to 14 years,female!!estimate!!age!!15 to 19 years,female!!estimate!!age!!20 to 24 years,female!!estimate!!age!!25 to 29 years,female!!estimate!!age!!30 to 34 years,female!!estimate!!age!!35 to 39 years,female!!estimate!!age!!40 to 44 years,female!!estimate!!age!!45 to 49 years,female!!estimate!!age!!50 to 54 years,female!!estimate!!age!!55 to 59 years,female!!estimate!!age!!60 to 64 years,female!!estimate!!age!!65 to 69 years,female!!estimate!!age!!70 to 74 years,female!!estimate!!age!!75 to 79 years,female!!estimate!!age!!80 to 84 years,

In [ ]:
age.columns = age.columns.str.split("total!!estimate!!").str[-1]
age_columns = [col for col in age.columns if col == "district" or col == "summary indicators!!median age (years)" or
               col.startswith("age!!")]

In [ ]:
age.columns

Index(['district', 'total population', 'age!!under 5 years',
       'age!!5 to 9 years', 'age!!10 to 14 years', 'age!!15 to 19 years',
       'age!!20 to 24 years', 'age!!25 to 29 years', 'age!!30 to 34 years',
       'age!!35 to 39 years',
       ...
       'female!!estimate!!selected age categories!!62 years and over',
       'female!!estimate!!selected age categories!!65 years and over',
       'female!!estimate!!selected age categories!!75 years and over',
       'female!!estimate!!summary indicators!!median age (years)',
       'female!!estimate!!summary indicators!!sex ratio (males per 100 females)',
       'female!!estimate!!summary indicators!!age dependency ratio',
       'female!!estimate!!summary indicators!!old-age dependency ratio',
       'female!!estimate!!summary indicators!!child dependency ratio',
       'female!!estimate!!percent allocated!!sex',
       'female!!estimate!!percent allocated!!age'],
      dtype='object', length=109)

In [ ]:
age = age[age_columns]
age.head()

,district,age!!under 5 years,age!!5 to 9 years,age!!10 to 14 years,age!!15 to 19 years,age!!20 to 24 years,age!!25 to 29 years,age!!30 to 34 years,age!!35 to 39 years,age!!40 to 44 years,age!!45 to 49 years,age!!50 to 54 years,age!!55 to 59 years,age!!60 to 64 years,age!!65 to 69 years,age!!70 to 74 years,age!!75 to 79 years,age!!80 to 84 years,age!!85 years and over,summary indicators!!median age (years)
1,"State House District 1 (2016), Alaska",6.9,6.8,5.2,4.1,13.8,10.1,7.8,6.2,5.2,4.2,5.9,7.0,6.2,2.8,2.6,1.9,1.7,1.4,31.8
2,"State House District 2 (2016), Alaska",12.3,8.5,5.7,6.7,16.5,13.6,8.6,5.9,4.1,4.1,3.5,3.6,1.8,2.2,1.3,0.8,0.6,0.3,25.2
3,"State House District 3 (2016), Alaska",8.6,8.8,6.4,7.7,6.5,8.1,8.3,5.9,7.0,7.3,7.7,5.5,5.6,3.2,1.9,0.4,0.5,0.6,31.7
4,"State House District 4 (2016), Alaska",6.0,6.1,5.4,5.5,8.1,10.4,8.7,6.1,6.4,5.5,5.5,6.6,9.5,5.3,2.3,0.9,1.0,0.6,34.9
5,"State House District 5 (2016), Alaska",5.8,5.4,6.2,9.1,11.3,7.6,6.3,8.1,6.3,6.8,8.9,6.4,3.9,3.1,2.5,1.5,0.5,0.4,33.7


In [ ]:
age.iloc[:, 1:] = age.iloc[:, 1:].astype("float")

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [ ]:
age["under 20"] = age.iloc[:, 1:5].sum(axis=1)
age["20 to 34"] = age.iloc[:, 5:8].sum(axis=1)
age["35 to 49"] = age.iloc[:, 8:11].sum(axis=1)
age["50 to 64"] = age.iloc[:, 11:14].sum(axis=1)
age["65 and over"] = age.iloc[:, 14:19].sum(axis=1)
age["median age"] = age.iloc[:, 19]
age.drop(age.iloc[:, 1:20].columns, axis=1, inplace=True)
age.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,district,under 20,20 to 34,35 to 49,50 to 64,65 and over,median age
1,"State House District 1 (2016), Alaska",23.0,31.7,15.6,19.1,10.4,31.8
2,"State House District 2 (2016), Alaska",33.2,38.7,14.1,8.9,5.2,25.2
3,"State House District 3 (2016), Alaska",31.5,22.9,20.2,18.8,6.6,31.7
4,"State House District 4 (2016), Alaska",23.0,27.2,18.0,21.6,10.1,34.9
5,"State House District 5 (2016), Alaska",26.5,25.2,21.2,19.2,8.0,33.7


#### Final Cleaning and Ditrict dropping

In [ ]:
age = clean_census_data(age, percentage=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [ ]:
age = drop_spare_districts(target_df, age)

#### Reset Index and Save DataFrame

In [ ]:
age = reset_and_save(age, "age")

In [ ]:
age.tail()

,district,under 20,20 to 34,35 to 49,50 to 64,65 and over,median age,state
3816,56,23.0,20.3,18.0,24.8,13.9,40.2,wyoming
3817,57,29.4,24.5,18.8,14.8,12.4,32.1,wyoming
3818,58,24.4,22.6,19.0,22.3,11.7,37.5,wyoming
3819,59,26.5,24.3,20.2,16.6,12.5,34.6,wyoming
3820,60,29.8,20.4,21.2,19.2,9.5,34.9,wyoming


## Sex

* sex = percentage of voting age female on total voting age population

In [ ]:
sex = pd.read_csv("drive/MyDrive/US Elections/Census Data/Sex 2016/sex_2016.csv")
sex.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,68,70,72,73,74,75,76,77,78,79,80,81,82,83,84,86,88,90,92,94,96,97,98,100,101,102,103,104,105,106,107,108,110,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,258,260,262,264,265,266,267,268,26

(3888, 338)

#### First Cleaning

In [ ]:
sex = first_clean(sex)
sex.head(1)

,estimate!!sex and age!!total population,percent!!sex and age!!total population,estimate!!sex and age!!total population!!male,percent!!sex and age!!total population!!male,estimate!!sex and age!!total population!!female,percent!!sex and age!!total population!!female,estimate!!sex and age!!under 5 years,percent!!sex and age!!under 5 years,estimate!!sex and age!!5 to 9 years,percent!!sex and age!!5 to 9 years,estimate!!sex and age!!10 to 14 years,percent!!sex and age!!10 to 14 years,estimate!!sex and age!!15 to 19 years,percent!!sex and age!!15 to 19 years,estimate!!sex and age!!20 to 24 years,percent!!sex and age!!20 to 24 years,estimate!!sex and age!!25 to 34 years,percent!!sex and age!!25 to 34 years,estimate!!sex and age!!35 to 44 years,percent!!sex and age!!35 to 44 years,estimate!!sex and age!!45 to 54 years,percent!!sex and age!!45 to 54 years,estimate!!sex and age!!55 to 59 years,percent!!sex and age!!55 to 59 years,estimate!!sex and age!!60 to 64 years,percent!!sex and age!!60 to 64 years,estimate!!sex and age!!65 to 74 years,percent!!sex and age!!65 to 74 years,estimate!!sex and age!!75 to 84 years,percent!!sex and age!!75 to 84 years,estimate!!sex and age!!85 years and over,percent!!sex and age!!85 years and over,estimate!!sex and age!!median age (years),percent!!sex and age!!median age (years),estimate!!sex and age!!18 years and over,percent!!sex and age!!18 years and over,estimate!!sex and age!!21 years and over,percent!!sex and age!!21 years and over,estimate!!sex and age!!62 years and over,percent!!sex and age!!62 years and over,estimate!!sex and age!!65 years and over,percent!!sex and age!!65 years and over,estimate!!sex and age!!18 years and over,percent!!sex and age!!18 years and over,estimate!!sex and age!!18 years and over!!male,percent!!sex and age!!18 years and over!!male,estimate!!sex and age!!18 years and over!!female,percent!!sex and age!!18 years and over!!female,estimate!!sex and age!!65 years and over,percent!!sex and age!!65 years and over,estimate!!sex and age!!65 years and over!!male,percent!!sex and age!!65 years and over!!male,estimate!!sex and age!!65 years and over!!female,percent!!sex and age!!65 years and over!!female,estimate!!race!!total population,percent!!race!!total population,estimate!!race!!total population!!one race,percent!!race!!total population!!one race,estimate!!race!!total population!!two or more races,percent!!race!!total population!!two or more races,estimate!!race!!one race,percent!!race!!one race,estimate!!race!!one race!!white,percent!!race!!one race!!white,estimate!!race!!one race!!black or african american,percent!!race!!one race!!black or african american,estimate!!race!!one race!!american indian and alaska native,percent!!race!!one race!!american indian and alaska native,estimate!!race!!one race!!american indian and alaska native!!cherokee tribal grouping,percent!!race!!one race!!american indian and alaska native!!cherokee tribal grouping,estimate!!race!!one race!!american indian and alaska native!!chippewa tribal grouping,percent!!race!!one race!!american indian and alaska native!!chippewa tribal grouping,estimate!!race!!one race!!american indian and alaska native!!navajo tribal grouping,percent!!race!!one race!!american indian and alaska native!!navajo tribal grouping,estimate!!race!!one race!!american indian and alaska native!!sioux tribal grouping,percent!!race!!one race!!american indian and alaska native!!sioux tribal grouping,estimate!!race!!one race!!asian,percent!!race!!one race!!asian,estimate!!race!!one race!!asian!!asian indian,percent!!race!!one race!!asian!!asian indian,estimate!!race!!one race!!asian!!chinese,percent!!race!!one race!!asian!!chinese,estimate!!race!!one race!!asian!!filipino,percent!!race!!one race!!asian!!filipino,estimate!!race!!one race!!asian!!japanese,percent!!race!!one race!!asian!!japanese,estimate!!race!!one race!!asian!!korean,percent!!race!!one race!!asian!!korean,estimate!!race!!one race!!asian!!vietnamese,percent!!race!!one race!!

#### DataFrame dependent Cleaning

In [ ]:
sex_columns = [col for col in sex.columns if col.startswith("estimate!!citizen, voting age population") or col == "district"]
sex = sex[sex_columns]
sex.head(1)

,"estimate!!citizen, voting age population!!citizen, 18 and over population","estimate!!citizen, voting age population!!citizen, 18 and over population!!male","estimate!!citizen, voting age population!!citizen, 18 and over population!!female",district
1,13629,7148,6481,"State House District 1 (2016), Alaska"


In [ ]:
sex = pd.concat([sex.district, sex.iloc[:, 1:-1]], axis=1)

In [ ]:
sex.columns = ["district", "percent " + sex.columns[1].split("!!")[-1], "percent " + sex.columns[2].split("!!")[-1]]

#### Final Cleaning and Ditrict dropping

In [ ]:
sex = clean_census_data(sex)

In [ ]:
sex = drop_spare_districts(target_df, sex)
sex.shape

(3821, 4)

In [ ]:
sex.drop(sex.columns[1], axis=1, inplace=True)

#### Reset Index and Save DataFrame

In [ ]:
sex = reset_and_save(sex, "sex")
sex.tail()

,district,percent female,state
3816,56,0.4685,wyoming
3817,57,0.5205,wyoming
3818,58,0.4612,wyoming
3819,59,0.5261,wyoming
3820,60,0.5021,wyoming


## Race

* race = 

In [ ]:
race = pd.read_csv("/content/drive/MyDrive/US Elections/Census Data/Race 2016/race_2016.csv")
race.shape

(3888, 22)

In [ ]:
race.head()

,B02001_001E,B02001_001M,B02001_002E,B02001_002M,B02001_003E,B02001_003M,B02001_004E,B02001_004M,B02001_005E,B02001_005M,B02001_006E,B02001_006M,B02001_007E,B02001_007M,B02001_008E,B02001_008M,B02001_009E,B02001_009M,B02001_010E,B02001_010M,GEO_ID,NAME
0,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!White alone,Margin of Error!!Total!!White alone,Estimate!!Total!!Black or African American alone,Margin of Error!!Total!!Black or African Ameri...,Estimate!!Total!!American Indian and Alaska Na...,Margin of Error!!Total!!American Indian and Al...,Estimate!!Total!!Asian alone,Margin of Error!!Total!!Asian alone,Estimate!!Total!!Native Hawaiian and Other Pac...,Margin of Error!!Total!!Native Hawaiian and Ot...,Estimate!!Total!!Some other race alone,Margin of Error!!Total!!Some other race alone,Estimate!!Total!!Two or more races,Margin of Error!!Total!!Two or more races,Estimate!!Total!!Two or more races!!Two races ...,Margin of Error!!Total!!Two or more races!!Two...,Estimate!!Total!!Two or more races!!Two races ...,Margin of Error!!Total!!Two or more races!!Two...,id,Geographic Area Name
1,18129,760,11431,687,1609,323,1953,314,829,287,179,155,202,152,1926,474,0,14,1926,474,620L500US02001,"State House District 1 (2016), Alaska"
2,18398,1151,13622,1193,1160,323,490,208,844,258,248,155,347,220,1687,355,10,16,1677,351,620L500US02002,"State House District 2 (2016), Alaska"
3,18831,1187,15470,1195,421,224,1394,656,170,107,0,14,43,49,1333,461,11,17,1322,459,620L500US02003,"State House District 3 (2016), Alaska"
4,19011,1390,17070,1300,46,51,717,337,265,196,45,64,34,63,834,359,9,16,825,359,620L500US02004,"State House District 4 (2016), Alaska"


#### First Cleaning

In [ ]:
race = first_clean(race)

In [ ]:
race.head(1)

,estimate!!total,estimate!!total!!white alone,estimate!!total!!black or african american alone,estimate!!total!!american indian and alaska native alone,estimate!!total!!asian alone,estimate!!total!!native hawaiian and other pacific islander alone,estimate!!total!!some other race alone,estimate!!total!!two or more races,estimate!!total!!two or more races!!two races including some other race,"estimate!!total!!two or more races!!two races excluding some other race, and three or more races",district
1,18129,11431,1609,1953,829,179,202,1926,0,1926,"State House District 1 (2016), Alaska"


#### DataFrame dependent Cleaning

In [ ]:
# Select columns
race = race.drop(race.iloc[:, -3:-1].columns, axis=1)
race.head(1)

,estimate!!total,estimate!!total!!white alone,estimate!!total!!black or african american alone,estimate!!total!!american indian and alaska native alone,estimate!!total!!asian alone,estimate!!total!!native hawaiian and other pacific islander alone,estimate!!total!!some other race alone,estimate!!total!!two or more races,district
1,18129,11431,1609,1953,829,179,202,1926,"State House District 1 (2016), Alaska"


In [ ]:
race.iloc[:, :-1] = race.iloc[:, :-1].astype("int")
race["non-white"] = race.iloc[:, 2:-1].sum(axis=1)
race.columns = race.columns.str.split("estimate!!total!!").str[-1]
district = race.district

In [ ]:
race.columns

Index(['estimate!!total', 'white alone', 'black or african american alone',
       'american indian and alaska native alone', 'asian alone',
       'native hawaiian and other pacific islander alone',
       'some other race alone', 'two or more races', 'district', 'non-white'],
      dtype='object')

In [ ]:
race_columns = ['estimate!!total', 'non-white', 'black or african american alone']
race = pd.concat([district, race[race_columns]], axis=1)
race.head()

,district,estimate!!total,non-white,black or african american alone
1,"State House District 1 (2016), Alaska",18129,6698.0,1609
2,"State House District 2 (2016), Alaska",18398,4776.0,1160
3,"State House District 3 (2016), Alaska",18831,3361.0,421
4,"State House District 4 (2016), Alaska",19011,1941.0,46
5,"State House District 5 (2016), Alaska",18072,5188.0,739


In [ ]:
race.iloc[:, 1:] = race.iloc[:, 1:].astype("float")
race["non-white"] = race["non-white"] / race["estimate!!total"]
race["black/afroamerican"] = race["black or african american alone"] / race["estimate!!total"]

In [ ]:
race = race.drop(race.columns[[1, 3]], axis=1)
race.head()

,district,non-white,black/afroamerican
1,"State House District 1 (2016), Alaska",0.369463,0.088753
2,"State House District 2 (2016), Alaska",0.259593,0.063050
3,"State House District 3 (2016), Alaska",0.178482,0.022357
4,"State House District 4 (2016), Alaska",0.102099,0.002420
5,"State House District 5 (2016), Alaska",0.287074,0.040892


#### Final Cleaning and Ditrict dropping

In [ ]:
race = clean_census_data(race, percentage=False)
race.shape

(3887, 4)

In [ ]:
race = drop_spare_districts(target_df, race)
race.tail()

,district,non-white,black/afroamerican,state
3883,56,0.112171,0.040680,wyoming
3884,57,0.045366,0.000308,wyoming
3885,58,0.086408,0.012723,wyoming
3886,59,0.078628,0.000742,wyoming
3887,60,0.078362,0.002439,wyoming


#### Reset Index and Save DataFrame

In [ ]:
race = reset_and_save(race, "race")
race.tail()

,district,non-white,black/afroamerican,state
3816,56,0.112171,0.040680,wyoming
3817,57,0.045366,0.000308,wyoming
3818,58,0.086408,0.012723,wyoming
3819,59,0.078628,0.000742,wyoming
3820,60,0.078362,0.002439,wyoming


## Health Insurance

* health_insurance = percent of uninsured people per district

In [ ]:
health_insurance = pd.read_csv("drive/MyDrive/US Elections/Census Data/Health Insurance 2016/health_insurance_2016.csv")
health_insurance.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,2

,GEO_ID,NAME,S2701_C01_001E,S2701_C01_001M,S2701_C01_002E,S2701_C01_002M,S2701_C01_003E,S2701_C01_003M,S2701_C01_004E,S2701_C01_004M,S2701_C01_005E,S2701_C01_005M,S2701_C01_006E,S2701_C01_006M,S2701_C01_007E,S2701_C01_007M,S2701_C01_008E,S2701_C01_008M,S2701_C01_009E,S2701_C01_009M,S2701_C01_010E,S2701_C01_010M,S2701_C01_011E,S2701_C01_011M,S2701_C01_012E,S2701_C01_012M,S2701_C01_013E,S2701_C01_013M,S2701_C01_014E,S2701_C01_014M,S2701_C01_015E,S2701_C01_015M,S2701_C01_016E,S2701_C01_016M,S2701_C01_017E,S2701_C01_017M,S2701_C01_018E,S2701_C01_018M,S2701_C01_019E,S2701_C01_019M,S2701_C01_020E,S2701_C01_020M,S2701_C01_021E,S2701_C01_021M,S2701_C01_022E,S2701_C01_022M,S2701_C01_023E,S2701_C01_023M,S2701_C01_024E,S2701_C01_024M,S2701_C01_025E,S2701_C01_025M,S2701_C01_026E,S2701_C01_026M,S2701_C01_027E,S2701_C01_027M,S2701_C01_028E,S2701_C01_028M,S2701_C01_029E,S2701_C01_029M,S2701_C01_030E,S2701_C01_030M,S2701_C01_031E,S2701_C01_031M,S2701_C01_032E,S2701_C01_032M,S2701_C01_033E,S2701_C01_033M,S2701_C01_034E,S2701_C01_034M,S2701_C01_035E,S2701_C01_035M,S2701_C01_036E,S2701_C01_036M,S2701_C01_037E,S2701_C01_037M,S2701_C01_038E,S2701_C01_038M,S2701_C01_039E,S2701_C01_039M,S2701_C01_040E,S2701_C01_040M,S2701_C01_041E,S2701_C01_041M,S2701_C01_042E,S2701_C01_042M,S2701_C01_043E,S2701_C01_043M,S2701_C01_044E,S2701_C01_044M,S2701_C01_045E,S2701_C01_045M,S2701_C01_046E,S2701_C01_046M,S2701_C01_047E,S2701_C01_047M,S2701_C01_048E,S2701_C01_048M,S2701_C01_049E,S2701_C01_049M,S2701_C01_050E,S2701_C01_050M,S2701_C01_051E,S2701_C01_051M,S2701_C01_052E,S2701_C01_052M,S2701_C01_053E,S2701_C01_053M,S2701_C01_054E,S2701_C01_054M,S2701_C01_055E,S2701_C01_055M,S2701_C01_056E,S2701_C01_056M,S2701_C01_057E,S2701_C01_057M,S2701_C01_058E,S2701_C01_058M,S2701_C01_059E,S2701_C01_059M,S2701_C01_060E,S2701_C01_060M,S2701_C01_061E,S2701_C01_061M,S2701_C01_062E,S2701_C01_062M,S2701_C02_001E,S2701_C02_001M,S2701_C02_002E,S2701_C02_002M,S2701_C02_003E,S2701_C02_003M,S2701_C02_004E,S2701_C02_004M,S2701_C02_005E,S2701_C02_005M,S2701_C02_006E,S2701_C02_006M,S2701_C02_007E,S2701_C02_007M,S2701_C02_008E,S2701_C02_008M,S2701_C02_009E,S2701_C02_009M,S2701_C02_010E,S2701_C02_010M,S2701_C02_011E,S2701_C02_011M,S2701_C02_012E,S2701_C02_012M,S2701_C02_013E,S2701_C02_013M,S2701_C02_014E,S2701_C02_014M,S2701_C02_015E,S2701_C02_015M,S2701_C02_016E,S2701_C02_016M,S2701_C02_017E,S2701_C02_017M,S2701_C02_018E,S2701_C02_018M,S2701_C02_019E,S2701_C02_019M,S2701_C02_020E,S2701_C02_020M,S2701_C02_021E,S2701_C02_021M,S2701_C02_022E,S2701_C02_022M,S2701_C02_023E,S2701_C02_023M,S2701_C02_024E,S2701_C02_024M,S2701_C02_025E,S2701_C02_025M,S2701_C02_026E,S2701_C02_026M,S2701_C02_027E,S2701_C02_027M,S2701_C02_028E,S2701_C02_028M,S2701_C02_029E,S2701_C02_029M,S2701_C02_030E,S2701_C02_030M,S2701_C02_031E,S2701_C02_031M,S2701_C02_032E,S2701_C02_032M,S2701_C02_033E,S2701_C02_033M,S2701_C02_034E,S2701_C02_034M,S2701_C02_035E,S2701_C02_035M,S2701_C02_036E,S2701_C02_036M,S2701_C02_037E,S2701_C02_037M,S2701_C02_038E,S2701_C02_038M,S2701_C02_039E,S2701_C02_039M,S2701_C02_040E,S2701_C02_040M,S2701_C02_041E,S2701_C02_041M,S2701_C02_042E,S2701_C02_042M,S2701_C02_043E,S2701_C02_043M,S2701_C02_044E,S2701_C02_044M,S2701_C02_045E,S2701_C02_045M,S2701_C02_046E,S2701_C02_046M,S2701_C02_047E,S2701_C02_047M,S2701_C02_048E,S2701_C02_048M,S2701_C02_049E,S2701_C02_049M,S2701_C02_050E,S2701_C02_050M,S2701_C02_051E,S2701_C02_051M,S2701_C02_052E,S2701_C02_052M,S2701_C02_053E,S2701_C02_053M,S2701_C02_054E,S2701_C02_054M,S2701_C02_055E,S2701_C02_055M,S2701_C02_056E,S2701_C02_056M,S2701_C02_057E,S2701_C02_057M,S2701_C02_058E,S2701_C02_058M,S2701_C02_059E,S2701_C02_059M,S2701_C02_060E,S2701_C02_060M,S2701_C02_061E,S2701_C02_061M,S2701_C02_062E,S2701_C02_062M,S2701_C03_001E,S2701_C03_001M,S2701_C03_002E,S2701_C03_002M,S2701_C03_003E,S2701_C03_003M,S2701_C03_004E,S2701_C03_004M,S2701_C03_005E,S2701_C03_005M,S2701_C03_006E,S2701_C03_006M,S2701_C03_007E,S2701_C03_007M,S2701_C03_008E,S2701_C03_008M,S2701_C03_009E,S2701_C03_00

In [ ]:
health_insurance.shape

(3888, 622)

#### First Cleaning

In [ ]:
health_insurance = first_clean(health_insurance)
health_insurance.head(1)

,district,total!!estimate!!civilian noninstitutionalized population,total!!estimate!!age!!under 18 years,total!!estimate!!age!!under 18 years!!under 6 years,total!!estimate!!age!!under 18 years!!6 to 17 years,total!!estimate!!age!!18 to 64 years,total!!estimate!!age!!18 to 64 years!!18 to 24 years,total!!estimate!!age!!18 to 64 years!!25 to 34 years,total!!estimate!!age!!18 to 64 years!!35 to 44 years,total!!estimate!!age!!18 to 64 years!!45 to 54 years,total!!estimate!!age!!18 to 64 years!!55 to 64 years,total!!estimate!!age!!65 years and older,total!!estimate!!age!!65 years and older!!65 to 74 years,total!!estimate!!age!!65 years and older!!75 years and older,total!!estimate!!19 to 25 years,total!!estimate!!sex!!male,total!!estimate!!sex!!female,total!!estimate!!race and hispanic or latino origin!!white alone,total!!estimate!!race and hispanic or latino origin!!black or african american alone,total!!estimate!!race and hispanic or latino origin!!american indian and alaska native alone,total!!estimate!!race and hispanic or latino origin!!asian alone,total!!estimate!!race and hispanic or latino origin!!native hawaiian and other pacific islander alone,total!!estimate!!race and hispanic or latino origin!!some other race alone,total!!estimate!!race and hispanic or latino origin!!two or more races,total!!estimate!!hispanic or latino (of any race),"total!!estimate!!white alone, not hispanic or latino",total!!estimate!!living arrangements!!in family households,total!!estimate!!living arrangements!!in family households!!in married couple families,total!!estimate!!living arrangements!!in family households!!in other families,"total!!estimate!!living arrangements!!in family households!!in other families!!male householder, no wife present","total!!estimate!!living arrangements!!in family households!!in other families!!female householder, no husband present",total!!estimate!!living arrangements!!in non-family households and other living arrangements,total!!estimate!!nativity and u.s. citizenship status!!native born,total!!estimate!!nativity and u.s. citizenship status!!foreign born,total!!estimate!!nativity and u.s. citizenship status!!foreign born!!naturalized,total!!estimate!!nativity and u.s. citizenship status!!foreign born!!not a citizen,total!!estimate!!disability status!!with a disability,total!!estimate!!disability status!!no disability,total!!estimate!!educational attainment!!civilian noninstitutionalized population 25 years and over,total!!estimate!!educational attainment!!civilian noninstitutionalized population 25 years and over!!less than high school graduate,total!!estimate!!educational attainment!!civilian noninstitutionalized population 25 years and over!!high school graduate (includes equivalency),total!!estimate!!educational attainment!!civilian noninstitutionalized population 25 years and over!!some college or associate's degree,total!!estimate!!educational attainment!!civilian noninstitutionalized population 25 years and over!!bachelor's degree or higher,total!!estimate!!employment status!!civilian noninstitutionalized population 18 years and over,total!!estimate!!employment status!!civilian noninstitutionalized population 18 years and over!!in labor force,total!!estimate!!employment status!!civilian noninstitutionalized population 18 years and over!!in labor force!!employed,total!!estimate!!employment status!!civilian noninstitutionalized population 18 years and over!!in labor force!!unemployed,total!!estimate!!employment status!!civilian noninstitutionalized population 18 years and over!!not in labor force,total!!estimate!!work experience!!civilian noninstitutionalized population 18 to 64 years,"total!!estimate!!work experience!!civilian noninstitutionalized population 18 to 64 years!!worked full-time, year round in the past 12 months","total!!estimate!!work experience!!civilian noninstitutionalized population 18 to 64 years!!worked less than full-time, year round in the past 12 months",total!!estimate!!work experienc

#### DataFrame dependent Cleaning

In [ ]:
# Select relevant columns
civilian_columns = [col for col in health_insurance.columns if "civilian noninstitutionalized population" in col or col == "district"]

In [ ]:
true_columns = health_insurance[civilian_columns].columns.str.split("!!").str.len() <= 3

In [ ]:
final_columns = health_insurance[civilian_columns].columns[true_columns]
health_insurance = health_insurance[final_columns]
health_insurance.head(1)

,district,total!!estimate!!civilian noninstitutionalized population,insured!!estimate!!civilian noninstitutionalized population,percent insured!!estimate!!civilian noninstitutionalized population,uninsured!!estimate!!civilian noninstitutionalized population,percent uninsured!!estimate!!civilian noninstitutionalized population
1,"State House District 1 (2016), Alaska",17256,14092,81.7,3164,18.3


In [ ]:
health_insurance.drop(health_insurance.iloc[:, 1:-1].columns, axis=1, inplace=True)
health_insurance.head(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,district,percent uninsured!!estimate!!civilian noninstitutionalized population
1,"State House District 1 (2016), Alaska",18.3


In [ ]:
health_insurance.columns = ["district", health_insurance.columns[1].split("!!")[0]]

#### Final Cleaning and Ditrict dropping

In [ ]:
health_insurance = clean_census_data(health_insurance, percentage=False)

In [ ]:
health_insurance = drop_spare_districts(target_df, health_insurance)

#### Reset Index and Save DataFrame

In [ ]:
health_insurance = reset_and_save(health_insurance, "health_insurance")
health_insurance.tail()

,district,percent uninsured,state
3816,56,16.8,wyoming
3817,57,14.6,wyoming
3818,58,17.7,wyoming
3819,59,20.9,wyoming
3820,60,12.6,wyoming


## Language Spoken at Home

In [ ]:
language_spoken = pd.read_csv("drive/MyDrive/US Elections/Census Data/Language Spoken at Home 2016/language_spoken_at_home_2016.csv")
language_spoken.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,90,91,92,93,94,95,96,97,98,99,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,150,151,184,185,188,189,192,193,194,195,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,236,237,238,239,240,241,242,243,246,247,280,281,284,285,288,289) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GEO_ID,NAME,S1601_C01_001E,S1601_C01_001M,S1601_C01_002E,S1601_C01_002M,S1601_C01_003E,S1601_C01_003M,S1601_C01_004E,S1601_C01_004M,S1601_C01_005E,S1601_C01_005M,S1601_C01_006E,S1601_C01_006M,S1601_C01_007E,S1601_C01_007M,S1601_C01_008E,S1601_C01_008M,S1601_C01_009E,S1601_C01_009M,S1601_C01_010E,S1601_C01_010M,S1601_C01_011E,S1601_C01_011M,S1601_C01_012E,S1601_C01_012M,S1601_C01_013E,S1601_C01_013M,S1601_C01_014E,S1601_C01_014M,S1601_C01_015E,S1601_C01_015M,S1601_C01_016E,S1601_C01_016M,S1601_C01_017E,S1601_C01_017M,S1601_C01_018E,S1601_C01_018M,S1601_C01_019E,S1601_C01_019M,S1601_C01_020E,S1601_C01_020M,S1601_C01_021E,S1601_C01_021M,S1601_C01_022E,S1601_C01_022M,S1601_C01_023E,S1601_C01_023M,S1601_C01_024E,S1601_C01_024M,S1601_C02_001E,S1601_C02_001M,S1601_C02_002E,S1601_C02_002M,S1601_C02_003E,S1601_C02_003M,S1601_C02_004E,S1601_C02_004M,S1601_C02_005E,S1601_C02_005M,S1601_C02_006E,S1601_C02_006M,S1601_C02_007E,S1601_C02_007M,S1601_C02_008E,S1601_C02_008M,S1601_C02_009E,S1601_C02_009M,S1601_C02_010E,S1601_C02_010M,S1601_C02_011E,S1601_C02_011M,S1601_C02_012E,S1601_C02_012M,S1601_C02_013E,S1601_C02_013M,S1601_C02_014E,S1601_C02_014M,S1601_C02_015E,S1601_C02_015M,S1601_C02_016E,S1601_C02_016M,S1601_C02_017E,S1601_C02_017M,S1601_C02_018E,S1601_C02_018M,S1601_C02_019E,S1601_C02_019M,S1601_C02_020E,S1601_C02_020M,S1601_C02_021E,S1601_C02_021M,S1601_C02_022E,S1601_C02_022M,S1601_C02_023E,S1601_C02_023M,S1601_C02_024E,S1601_C02_024M,S1601_C03_001E,S1601_C03_001M,S1601_C03_002E,S1601_C03_002M,S1601_C03_003E,S1601_C03_003M,S1601_C03_004E,S1601_C03_004M,S1601_C03_005E,S1601_C03_005M,S1601_C03_006E,S1601_C03_006M,S1601_C03_007E,S1601_C03_007M,S1601_C03_008E,S1601_C03_008M,S1601_C03_009E,S1601_C03_009M,S1601_C03_010E,S1601_C03_010M,S1601_C03_011E,S1601_C03_011M,S1601_C03_012E,S1601_C03_012M,S1601_C03_013E,S1601_C03_013M,S1601_C03_014E,S1601_C03_014M,S1601_C03_015E,S1601_C03_015M,S1601_C03_016E,S1601_C03_016M,S1601_C03_017E,S1601_C03_017M,S1601_C03_018E,S1601_C03_018M,S1601_C03_019E,S1601_C03_019M,S1601_C03_020E,S1601_C03_020M,S1601_C03_021E,S1601_C03_021M,S1601_C03_022E,S1601_C03_022M,S1601_C03_023E,S1601_C03_023M,S1601_C03_024E,S1601_C03_024M,S1601_C04_001E,S1601_C04_001M,S1601_C04_002E,S1601_C04_002M,S1601_C04_003E,S1601_C04_003M,S1601_C04_004E,S1601_C04_004M,S1601_C04_005E,S1601_C04_005M,S1601_C04_006E,S1601_C04_006M,S1601_C04_007E,S1601_C04_007M,S1601_C04_008E,S1601_C04_008M,S1601_C04_009E,S1601_C04_009M,S1601_C04_010E,S1601_C04_010M,S1601_C04_011E,S1601_C04_011M,S1601_C04_012E,S1601_C04_012M,S1601_C04_013E,S1601_C04_013M,S1601_C04_014E,S1601_C04_014M,S1601_C04_015E,S1601_C04_015M,S1601_C04_016E,S1601_C04_016M,S1601_C04_017E,S1601_C04_017M,S1601_C04_018E,S1601_C04_018M,S1601_C04_019E,S1601_C04_019M,S1601_C04_020E,S1601_C04_020M,S1601_C04_021E,S1601_C04_021M,S1601_C04_022E,S1601_C04_022M,S1601_C04_023E,S1601_C04_023M,S1601_C04_024E,S1601_C04_024M,S1601_C05_001E,S1601_C05_001M,S1601_C05_002E,S1601_C05_002M,S1601_C05_003E,S1601_C05_003M,S1601_C05_004E,S1601_C05_004M,S1601_C05_005E,S1601_C05_005M,S1601_C05_006E,S1601_C05_006M,S1601_C05_007E,S1601_C05_007M,S1601_C05_008E,S1601_C05_008M,S1601_C05_009E,S1601_C05_009M,S1601_C05_010E,S1601_C05_010M,S1601_C05_011E,S1601_C05_011M,S1601_C05_012E,S1601_C05_012M,S1601_C05_013E,S1601_C05_013M,S1601_C05_014E,S1601_C05_014M,S1601_C05_015E,S1601_C05_015M,S1601_C05_016E,S1601_C05_016M,S1601_C05_017E,S1601_C05_017M,S1601_C05_018E,S1601_C05_018M,S1601_C05_019E,S1601_C05_019M,S1601_C05_020E,S1601_C05_020M,S1601_C05_021E,S1601_C05_021M,S1601_C05_022E,S1601_C05_022M,S1601_C05_023E,S1601_C05_023M,S1601_C05_024E,S1601_C05_024M,S1601_C06_001E,S1601_C06_001M,S1601_C06_002E,S1601_C06_002M,S1601_C06_003E,S1601_C06_003M,S1601_C06_004E,S1601_C06_004M,S1601_C06_005E,S1601_C06_005M,S1601_C06_006E,S1601_C06_006M,S1601_C06_007E,S1601_C06_007M,S1601_C06_008E,S1601_C06_008M,S1601_C06_009E,S1601_C06_009M,S1601_C06_010E,S1601_C06_010M,S1601_C06_011E,S1601_C06_011M,S1601_C06_012E,S1601_C06_012M,S1601_C06_013E,S1601_C06_01

#### First Cleaning

In [ ]:
language_spoken = first_clean(language_spoken)
language_spoken.head(1)

,district,total!!estimate!!population 5 years and over,total!!estimate!!speak only english,total!!estimate!!speak a language other than english,total!!estimate!!speak a language other than english!!spanish,total!!estimate!!speak a language other than english!!spanish!!5 to 17 years old,total!!estimate!!speak a language other than english!!spanish!!18 to 64 years old,total!!estimate!!speak a language other than english!!spanish!!65 years old and over,total!!estimate!!speak a language other than english!!other indo-european languages,total!!estimate!!speak a language other than english!!other indo-european languages!!5 to 17 years old,total!!estimate!!speak a language other than english!!other indo-european languages!!18 to 64 years old,total!!estimate!!speak a language other than english!!other indo-european languages!!65 years old and over,total!!estimate!!speak a language other than english!!asian and pacific island languages,total!!estimate!!speak a language other than english!!asian and pacific island languages!!5 to 17 years old,total!!estimate!!speak a language other than english!!asian and pacific island languages!!18 to 64 years old,total!!estimate!!speak a language other than english!!asian and pacific island languages!!65 years old and over,total!!estimate!!speak a language other than english!!other languages,total!!estimate!!speak a language other than english!!other languages!!5 to 17 years old,total!!estimate!!speak a language other than english!!other languages!!18 to 64 years old,total!!estimate!!speak a language other than english!!other languages!!65 years old and over,total!!estimate!!citizens 18 years and over!!all citizens 18 years old and over,total!!estimate!!citizens 18 years and over!!all citizens 18 years old and over!!speak only english,total!!estimate!!citizens 18 years and over!!all citizens 18 years old and over!!speak a language other than english,total!!estimate!!citizens 18 years and over!!all citizens 18 years old and over!!speak a language other than english!!spanish,total!!estimate!!citizens 18 years and over!!all citizens 18 years old and over!!speak a language other than english!!other languages,percent!!estimate!!population 5 years and over,percent!!estimate!!speak only english,percent!!estimate!!speak a language other than english,percent!!estimate!!speak a language other than english!!spanish,percent!!estimate!!speak a language other than english!!spanish!!5 to 17 years old,percent!!estimate!!speak a language other than english!!spanish!!18 to 64 years old,percent!!estimate!!speak a language other than english!!spanish!!65 years old and over,percent!!estimate!!speak a language other than english!!other indo-european languages,percent!!estimate!!speak a language other than english!!other indo-european languages!!5 to 17 years old,percent!!estimate!!speak a language other than english!!other indo-european languages!!18 to 64 years old,percent!!estimate!!speak a language other than english!!other indo-european languages!!65 years old and over,percent!!estimate!!speak a language other than english!!asian and pacific island languages,percent!!estimate!!speak a language other than english!!asian and pacific island languages!!5 to 17 years old,percent!!estimate!!speak a language other than english!!asian and pacific island languages!!18 to 64 years old,percent!!estimate!!speak a language other than english!!asian and pacific island languages!!65 years old and over,percent!!estimate!!speak a language other than english!!other languages,percent!!estimate!!speak a language other than english!!other languages!!5 to 17 years old,percent!!estimate!!speak a language other than english!!other languages!!18 to 64 years old,percent!!estimate!!speak a language other than english!!other languages!!65 years old and over,percent!!estimate!!citizens 18 years and over!!all citizens 18 years old and over,percent!!estimate!!citizens 18 years and over!!all citizens 18 years old and over!!speak only english,percent

#### DataFrame dependent Cleaning

In [ ]:
columns = [col for col in language_spoken.columns if "percent!!estimate!!speak a language other than english" in col or
           col == "district"]
language_spoken = language_spoken[columns].iloc[:, :2]
language_spoken.head()

,district,percent!!estimate!!speak a language other than english
1,"State House District 1 (2016), Alaska",15.4
2,"State House District 2 (2016), Alaska",15.0
3,"State House District 3 (2016), Alaska",6.4
4,"State House District 4 (2016), Alaska",6.5
5,"State House District 5 (2016), Alaska",10.0


In [ ]:
language_spoken.columns = ["district", "household language not english"]

#### Final Cleaning and Ditrict dropping

In [ ]:
language_spoken = clean_census_data(language_spoken, percentage=False)

In [ ]:
language_spoken = drop_spare_districts(target_df, language_spoken)

#### Reset Index and Save DataFrame

In [ ]:
language_spoken = reset_and_save(language_spoken, "language_spoken")
language_spoken.tail()

,district,household language not english,state
3816,56,5.4,wyoming
3817,57,4.8,wyoming
3818,58,5.9,wyoming
3819,59,8.8,wyoming
3820,60,13.5,wyoming


## Food Stamps

In [ ]:
food_stamps = pd.read_csv("drive/MyDrive/US Elections/Census Data/Food Stamps 2016/food_stamps_2016.csv")
food_stamps.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260

,GEO_ID,NAME,S2201_C01_001E,S2201_C01_001M,S2201_C01_002E,S2201_C01_002M,S2201_C01_003E,S2201_C01_003M,S2201_C01_004E,S2201_C01_004M,S2201_C01_005E,S2201_C01_005M,S2201_C01_006E,S2201_C01_006M,S2201_C01_007E,S2201_C01_007M,S2201_C01_008E,S2201_C01_008M,S2201_C01_009E,S2201_C01_009M,S2201_C01_010E,S2201_C01_010M,S2201_C01_011E,S2201_C01_011M,S2201_C01_012E,S2201_C01_012M,S2201_C01_013E,S2201_C01_013M,S2201_C01_014E,S2201_C01_014M,S2201_C01_015E,S2201_C01_015M,S2201_C01_016E,S2201_C01_016M,S2201_C01_017E,S2201_C01_017M,S2201_C01_018E,S2201_C01_018M,S2201_C01_019E,S2201_C01_019M,S2201_C01_020E,S2201_C01_020M,S2201_C01_021E,S2201_C01_021M,S2201_C01_022E,S2201_C01_022M,S2201_C01_023E,S2201_C01_023M,S2201_C01_024E,S2201_C01_024M,S2201_C01_025E,S2201_C01_025M,S2201_C01_026E,S2201_C01_026M,S2201_C01_027E,S2201_C01_027M,S2201_C01_028E,S2201_C01_028M,S2201_C01_029E,S2201_C01_029M,S2201_C01_030E,S2201_C01_030M,S2201_C01_031E,S2201_C01_031M,S2201_C01_032E,S2201_C01_032M,S2201_C01_033E,S2201_C01_033M,S2201_C01_034E,S2201_C01_034M,S2201_C01_035E,S2201_C01_035M,S2201_C01_036E,S2201_C01_036M,S2201_C01_037E,S2201_C01_037M,S2201_C01_038E,S2201_C01_038M,S2201_C02_001E,S2201_C02_001M,S2201_C02_002E,S2201_C02_002M,S2201_C02_003E,S2201_C02_003M,S2201_C02_004E,S2201_C02_004M,S2201_C02_005E,S2201_C02_005M,S2201_C02_006E,S2201_C02_006M,S2201_C02_007E,S2201_C02_007M,S2201_C02_008E,S2201_C02_008M,S2201_C02_009E,S2201_C02_009M,S2201_C02_010E,S2201_C02_010M,S2201_C02_011E,S2201_C02_011M,S2201_C02_012E,S2201_C02_012M,S2201_C02_013E,S2201_C02_013M,S2201_C02_014E,S2201_C02_014M,S2201_C02_015E,S2201_C02_015M,S2201_C02_016E,S2201_C02_016M,S2201_C02_017E,S2201_C02_017M,S2201_C02_018E,S2201_C02_018M,S2201_C02_019E,S2201_C02_019M,S2201_C02_020E,S2201_C02_020M,S2201_C02_021E,S2201_C02_021M,S2201_C02_022E,S2201_C02_022M,S2201_C02_023E,S2201_C02_023M,S2201_C02_024E,S2201_C02_024M,S2201_C02_025E,S2201_C02_025M,S2201_C02_026E,S2201_C02_026M,S2201_C02_027E,S2201_C02_027M,S2201_C02_028E,S2201_C02_028M,S2201_C02_029E,S2201_C02_029M,S2201_C02_030E,S2201_C02_030M,S2201_C02_031E,S2201_C02_031M,S2201_C02_032E,S2201_C02_032M,S2201_C02_033E,S2201_C02_033M,S2201_C02_034E,S2201_C02_034M,S2201_C02_035E,S2201_C02_035M,S2201_C02_036E,S2201_C02_036M,S2201_C02_037E,S2201_C02_037M,S2201_C02_038E,S2201_C02_038M,S2201_C03_001E,S2201_C03_001M,S2201_C03_002E,S2201_C03_002M,S2201_C03_003E,S2201_C03_003M,S2201_C03_004E,S2201_C03_004M,S2201_C03_005E,S2201_C03_005M,S2201_C03_006E,S2201_C03_006M,S2201_C03_007E,S2201_C03_007M,S2201_C03_008E,S2201_C03_008M,S2201_C03_009E,S2201_C03_009M,S2201_C03_010E,S2201_C03_010M,S2201_C03_011E,S2201_C03_011M,S2201_C03_012E,S2201_C03_012M,S2201_C03_013E,S2201_C03_013M,S2201_C03_014E,S2201_C03_014M,S2201_C03_015E,S2201_C03_015M,S2201_C03_016E,S2201_C03_016M,S2201_C03_017E,S2201_C03_017M,S2201_C03_018E,S2201_C03_018M,S2201_C03_019E,S2201_C03_019M,S2201_C03_020E,S2201_C03_020M,S2201_C03_021E,S2201_C03_021M,S2201_C03_022E,S2201_C03_022M,S2201_C03_023E,S2201_C03_023M,S2201_C03_024E,S2201_C03_024M,S2201_C03_025E,S2201_C03_025M,S2201_C03_026E,S2201_C03_026M,S2201_C03_027E,S2201_C03_027M,S2201_C03_028E,S2201_C03_028M,S2201_C03_029E,S2201_C03_029M,S2201_C03_030E,S2201_C03_030M,S2201_C03_031E,S2201_C03_031M,S2201_C03_032E,S2201_C03_032M,S2201_C03_033E,S2201_C03_033M,S2201_C03_034E,S2201_C03_034M,S2201_C03_035E,S2201_C03_035M,S2201_C03_036E,S2201_C03_036M,S2201_C03_037E,S2201_C03_037M,S2201_C03_038E,S2201_C03_038M,S2201_C04_001E,S2201_C04_001M,S2201_C04_002E,S2201_C04_002M,S2201_C04_003E,S2201_C04_003M,S2201_C04_004E,S2201_C04_004M,S2201_C04_005E,S2201_C04_005M,S2201_C04_006E,S2201_C04_006M,S2201_C04_007E,S2201_C04_007M,S2201_C04_008E,S2201_C04_008M,S2201_C04_009E,S2201_C04_009M,S2201_C04_010E,S2201_C04_010M,S2201_C04_011E,S2201_C04_011M,S2201_C04_012E,S2201_C04_012M,S2201_C04_013E,S2201_C04_013M,S2201_C04_014E,S2201_C04_014M,S2201_C04_015E,S2201_C04_015M,S2201_C04_016E,S2201_C04_016M,S2201_C04_017E,S2201_C04_017M,S2201_C04_018E,S2201_C04_018M,S2201_C04_019E,S2201_C04_01

#### First Cleaning

In [ ]:
food_stamps = first_clean(food_stamps)
food_stamps.head(1)

,district,total!!estimate!!households,total!!estimate!!with one or more people in the household 60 years and over,total!!estimate!!no people in the household 60 years and over,total!!estimate!!household type!!married-couple family,total!!estimate!!household type!!other family,"total!!estimate!!household type!!other family!!male householder, no wife present","total!!estimate!!household type!!other family!!female householder, no husband present",total!!estimate!!household type!!nonfamily households,total!!estimate!!household type!!with children under 18 years,total!!estimate!!household type!!with children under 18 years!!married-couple family,total!!estimate!!household type!!with children under 18 years!!other family,"total!!estimate!!household type!!with children under 18 years!!other family!!male householder, no wife present","total!!estimate!!household type!!with children under 18 years!!other family!!female householder, no husband present",total!!estimate!!household type!!with children under 18 years!!nonfamily households,total!!estimate!!household type!!no children under 18 years,total!!estimate!!household type!!no children under 18 years!!married-couple family,total!!estimate!!household type!!no children under 18 years!!other family,"total!!estimate!!household type!!no children under 18 years!!other family!!male householder, no wife present","total!!estimate!!household type!!no children under 18 years!!other family!!female householder, no husband present",total!!estimate!!household type!!no children under 18 years!!nonfamily households,total!!estimate!!poverty status in the past 12 months!!below poverty level,total!!estimate!!poverty status in the past 12 months!!at or above poverty level,total!!estimate!!disability status!!with one or more people with a disability,total!!estimate!!disability status!!with no persons with a disability,total!!estimate!!race and hispanic or latino origin of householder!!white alone,total!!estimate!!race and hispanic or latino origin of householder!!black or african american alone,total!!estimate!!race and hispanic or latino origin of householder!!american indian and alaska native alone,total!!estimate!!race and hispanic or latino origin of householder!!asian alone,total!!estimate!!race and hispanic or latino origin of householder!!native hawaiian and other pacific islander alone,total!!estimate!!race and hispanic or latino origin of householder!!some other race alone,total!!estimate!!race and hispanic or latino origin of householder!!two or more races,total!!estimate!!hispanic or latino origin (of any race),"total!!estimate!!white alone, not hispanic or latino",total!!estimate!!household income in the past 12 months (in 2016 inflation-adjusted dollars)!!median income (dollars),total!!estimate!!work status!!families,total!!estimate!!work status!!families!!no workers in past 12 months,total!!estimate!!work status!!families!!1 worker in past 12 months,total!!estimate!!work status!!families!!2 or more workers in past 12 months,percent!!estimate!!households,percent!!estimate!!with one or more people in the household 60 years and over,percent!!estimate!!no people in the household 60 years and over,percent!!estimate!!household type!!married-couple family,percent!!estimate!!household type!!other family,"percent!!estimate!!household type!!other family!!male householder, no wife present","percent!!estimate!!household type!!other family!!female householder, no husband present",percent!!estimate!!household type!!nonfamily households,percent!!estimate!!household type!!with children under 18 years,percent!!estimate!!household type!!with children under 18 years!!married-couple family,percent!!estimate!!household type!!with children under 18 years!!other family,"percent!!estimate!!household type!!with children under 18 years!!other family!!male householder, no wife present","percent!!estimate!!household type!!with children under 18 years!!other family!!female householder, no husband present",percent!!estimat

#### DataFrame dependent Cleaning

In [ ]:
food_stamps = food_stamps[["district", "percent households receiving food stamps/snap!!estimate!!households"]]
food_stamps.head()

,district,percent households receiving food stamps/snap!!estimate!!households
1,"State House District 1 (2016), Alaska",14.1
2,"State House District 2 (2016), Alaska",7.2
3,"State House District 3 (2016), Alaska",5.4
4,"State House District 4 (2016), Alaska",4.7
5,"State House District 5 (2016), Alaska",4.0


In [ ]:
food_stamps.rename(
    columns={"percent households receiving food stamps/snap!!estimate!!households": "percent households receiving food stamps"},
    inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Final Cleaning and Ditrict dropping

In [ ]:
food_stamps = clean_census_data(food_stamps, percentage=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
food_stamps = drop_spare_districts(target_df, food_stamps)
food_stamps.tail()

,district,percent households receiving food stamps,state
3883,56,12.6,wyoming
3884,57,6.0,wyoming
3885,58,8.2,wyoming
3886,59,9.1,wyoming
3887,60,4.1,wyoming


#### Reset Index and Save DataFrame

In [ ]:
food_stamps = reset_and_save(food_stamps, "food_stamps")
food_stamps.tail()

,district,percent households receiving food stamps,state
3816,56,12.6,wyoming
3817,57,6.0,wyoming
3818,58,8.2,wyoming
3819,59,9.1,wyoming
3820,60,4.1,wyoming


## Merge DataFrames

In [ ]:
import os

# Get all the Census Data Files 
path = "drive/MyDrive/US Elections/Census Data/Final Files"
list_of_dfs = os.listdir(path)[1:]
list_of_dfs = [file for file in list_of_dfs if "2016" in file]

# Read in target DataFrame
target_df = pd.read_csv("drive/MyDrive/US Elections/data_target_2016.csv")
df = target_df.copy() # copy to make following for loop work

In [ ]:
# Loop through files to merge them with target DataFrame
for file in list_of_dfs:
  census = pd.read_csv(path+"/"+file)
  df = pd.merge(left=census, right=df, how="inner", on=["district", "state"])

In [ ]:
df.head()

,district,percent households receiving food stamps,state,percent uninsured,household language not english,non-white,black/afroamerican,percent female,under 20,20 to 34,35 to 49,50 to 64,65 and over,median age,monthly housing costs - $499 or lower,monthly housing costs - $500 - 999,monthly housing costs - $1000 1999,monthly housing costs - $2000 or higher,monthly housing costs - median,percent owner-occupied housing units,average family size,native,foreign born,percent of tot. population in poverty,"management, business, science, and arts occupations",service occupations,sales and office occupations,blue collar occupations,earnings $1 - $24'999,earnings $25'000 - 49'999,earnings $50'000 - 74'999,earnings $75'000 or more,median earnings,less than highschool,highschool,college or associate,bachelor or higher,democrat,republican,other,year,target
0,1,14.1,alaska,18.3,15.4,0.369463,0.088753,0.4755,23.0,31.7,15.6,19.1,10.4,31.8,9.6,26.9,46.3,14.7,1165.0,42.3,3.34,0.9144,0.0856,15.3,0.2823,0.2360,0.2810,0.2008,0.1792,0.4074,0.2382,0.1752,30518,0.1055,0.3204,0.3830,0.1911,1.0,0.0,0.0,2016,1
1,2,7.2,alaska,8.1,15.0,0.259593,0.063050,0.4379,33.2,38.7,14.1,8.9,5.2,25.2,4.5,15.4,40.6,35.0,1674.0,36.9,3.43,0.9345,0.0655,6.4,0.3377,0.2091,0.2682,0.1850,0.2422,0.3664,0.2042,0.1872,31519,0.0500,0.3044,0.4334,0.2122,1153.0,3268.0,0.0,2016,0
2,3,5.4,alaska,11.0,6.4,0.178482,0.022357,0.4623,31.5,22.9,20.2,18.8,6.6,31.7,7.2,17.6,43.7,30.9,1680.0,74.4,3.48,0.9707,0.0293,7.4,0.3624,0.1202,0.2143,0.3031,0.1030,0.3200,0.3270,0.2500,38923,0.0589,0.2771,0.4483,0.2158,537.0,4291.0,2270.0,2016,0
3,4,4.7,alaska,8.2,6.5,0.102099,0.002420,0.4416,23.0,27.2,18.0,21.6,10.1,34.9,9.2,25.2,40.2,22.5,1328.0,76.4,3.19,0.9857,0.0143,5.1,0.4297,0.1777,0.1743,0.2184,0.0900,0.2560,0.2400,0.4140,45965,0.0462,0.1403,0.3673,0.4461,1.0,0.0,0.0,2016,1
4,5,4.0,alaska,14.1,10.0,0.287074,0.040892,0.4618,26.5,25.2,21.2,19.2,8.0,33.7,10.0,25.0,44.7,20.0,1242.0,64.7,3.46,0.9405,0.0595,5.9,0.4315,0.1362,0.2408,0.1915,0.0870,0.3300,0.2200,0.3630,36468,0.0534,0.1915,0.4128,0.3423,3812.0,3384.0,0.0,2016,1


In [ ]:
# Save DataFrame
df.to_csv("drive/MyDrive/US Elections/data_final_2016.csv", index=False)